In [1]:
# Big Multi-AGN Catalog (Big MAGN) DR1 - Dual Quasars/Candidates from quasar lens searches
# Author: R. W. Pfeifle
# Creation Date: 13 January 2022
# Last Revision: 5 January 2024

# Purpose: 
# 

In [2]:
# Load in packages for pandas, astropy, etc. 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.table import Table
from astropy.io import ascii
from astropy.table import Column, MaskedColumn
from astropy.io.ascii import masked
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.cosmology import LambdaCDM 
from astroquery.simbad import Simbad
from astroquery.sdss import SDSS
from astropy.coordinates import match_coordinates_sky
import os 

cosmo = LambdaCDM(H0=70, Om0=0.3, Ode0=0.7) #Creating our choice of cosmology here...

pd.set_option('display.max_columns', 300) # Setting max number of rows per df to be the size of the df
pd.set_option('display.max_rows', None)

In [3]:
def name_to_coords1(df,dfcol):
    if (len(dfcol[0])) == 18:
        df['Coordinates'] = dfcol.str.slice(start=5) # Stripping the J
        df['RA_test'] = df['Coordinates'].str.slice(start=0, stop=6) # Stripping the DEC parts 
        df['Dec_test'] = df['Coordinates'].str.slice(start=6, stop=13) # Stripping the RA parts
        df['RA1'] = df['RA_test'].str.slice(start=0, stop=2)+":"+df['RA_test'].str.slice(start=2, stop=4)+":"+df['RA_test'].str.slice(start=4, stop=6) # Putting together the RA coordinates separated by colons
        df['Dec1'] = df['Dec_test'].str.slice(start=0, stop=3)+":"+df['Dec_test'].str.slice(start=3, stop=5)+":"+df['Dec_test'].str.slice(start=5, stop=8) # Putting together the Dec coodinates separated by colons
        df.drop(columns=['Coordinates','RA_test','Dec_test'], inplace=True)
        return
    #print(dfcol.apply(len))
    elif (len(dfcol[0])) == 23:
        df['Coordinates'] = dfcol.str.slice(start=5) # Stripping the J
        df['RA_test'] = df['Coordinates'].str.slice(start=0, stop=9) # Stripping the DEC parts 
        df['Dec_test'] = df['Coordinates'].str.slice(start=9, stop=19) # Stripping the RA parts
        df['RA1'] = df['RA_test'].str.slice(start=0, stop=2)+":"+df['RA_test'].str.slice(start=2, stop=4)+":"+df['RA_test'].str.slice(start=4, stop=9) # Putting together the RA coordinates separated by colons
        df['Dec1'] = df['Dec_test'].str.slice(start=0, stop=3)+":"+df['Dec_test'].str.slice(start=3, stop=5)+":"+df['Dec_test'].str.slice(start=5, stop=10) # Putting together the Dec coodinates separated by colons
        df.drop(columns=['Coordinates','RA_test','Dec_test'], inplace=True)
        return
    #elif dfcol.apply(len) ==
    else:
        print('Error Encountered')
        
def name_to_coords2(df,dfcol):
    if (len(dfcol[0])) == 18:
        df['Coordinates'] = dfcol.str.slice(start=5) # Stripping the J
        df['RA_test'] = df['Coordinates'].str.slice(start=0, stop=6) # Stripping the DEC parts 
        df['Dec_test'] = df['Coordinates'].str.slice(start=6, stop=13) # Stripping the RA parts
        df['RA2'] = df['RA_test'].str.slice(start=0, stop=2)+":"+df['RA_test'].str.slice(start=2, stop=4)+":"+df['RA_test'].str.slice(start=4, stop=6) # Putting together the RA coordinates separated by colons
        df['Dec2'] = df['Dec_test'].str.slice(start=0, stop=3)+":"+df['Dec_test'].str.slice(start=3, stop=5)+":"+df['Dec_test'].str.slice(start=5, stop=8) # Putting together the Dec coodinates separated by colons
        df.drop(columns=['Coordinates','RA_test','Dec_test'], inplace=True)
        return
    #print(dfcol.apply(len))
    elif (len(dfcol[0])) == 23:
        df['Coordinates'] = dfcol.str.slice(start=5) # Stripping the J
        df['RA_test'] = df['Coordinates'].str.slice(start=0, stop=9) # Stripping the DEC parts 
        df['Dec_test'] = df['Coordinates'].str.slice(start=9, stop=19) # Stripping the RA parts
        df['RA2'] = df['RA_test'].str.slice(start=0, stop=2)+":"+df['RA_test'].str.slice(start=2, stop=4)+":"+df['RA_test'].str.slice(start=4, stop=9) # Putting together the RA coordinates separated by colons
        df['Dec2'] = df['Dec_test'].str.slice(start=0, stop=3)+":"+df['Dec_test'].str.slice(start=3, stop=5)+":"+df['Dec_test'].str.slice(start=5, stop=10) # Putting together the Dec coodinates separated by colons
        df.drop(columns=['Coordinates','RA_test','Dec_test'], inplace=True)
        return
    #elif dfcol.apply(len) ==
    else:
        print('Error Encountered')

def name_to_coords(df,dfcol):
    if (len(dfcol[0])) == 18:
        df['Coordinates'] = dfcol.str.slice(start=1) # Stripping the J
        df['RA_test'] = df['Coordinates'].str.slice(start=0, stop=6) # Stripping the DEC parts 
        df['Dec_test'] = df['Coordinates'].str.slice(start=6, stop=13) # Stripping the RA parts
        df['RA'] = df['RA_test'].str.slice(start=0, stop=2)+":"+df['RA_test'].str.slice(start=2, stop=4)+":"+df['RA_test'].str.slice(start=4, stop=6) # Putting together the RA coordinates separated by colons
        df['Dec'] = df['Dec_test'].str.slice(start=0, stop=3)+":"+df['Dec_test'].str.slice(start=3, stop=5)+":"+df['Dec_test'].str.slice(start=5, stop=8) # Putting together the Dec coodinates separated by colons
        df.drop(columns=['Coordinates','RA_test','Dec_test'], inplace=True)
        return
    #print(dfcol.apply(len))
    elif (len(dfcol[0])) == 23:
        df['Coordinates'] = dfcol.str.slice(start=5) # Stripping the J
        df['RA_test'] = df['Coordinates'].str.slice(start=0, stop=9) # Stripping the DEC parts 
        df['Dec_test'] = df['Coordinates'].str.slice(start=9, stop=19) # Stripping the RA parts
        df['RA'] = df['RA_test'].str.slice(start=0, stop=2)+":"+df['RA_test'].str.slice(start=2, stop=4)+":"+df['RA_test'].str.slice(start=4, stop=9) # Putting together the RA coordinates separated by colons
        df['Dec'] = df['Dec_test'].str.slice(start=0, stop=3)+":"+df['Dec_test'].str.slice(start=3, stop=5)+":"+df['Dec_test'].str.slice(start=5, stop=10) # Putting together the Dec coodinates separated by colons
        df.drop(columns=['Coordinates','RA_test','Dec_test'], inplace=True)
        return
    #elif dfcol.apply(len) ==
    else:
        print('Error Encountered')


In [4]:

## Convert Ra and Dec from degrees to sexagesimal format
#coords = SkyCoord(ra=df['ra']*u.degree, dec=df['dec']*u.degree, frame='icrs')
#df['ra_sexagesimal'] = coords.ra.to_string(u.hour, sep=':', precision=2)
#df['dec_sexagesimal'] = coords.dec.to_string(u.deg, sep=':', precision=2)
#
## Convert RA and Dec to the desired format
#ra_format = coords.ra.to_string(unit=u.hour, sep='', precision=2, pad=True)
#dec_format = coords.dec.to_string(unit=u.deg, sep='', precision=2, alwayssign=True, pad=True)
#
## Concatenate to form J_format using a loop
#df['J_format'] = ["J" + ra + dec for ra, dec in zip(ra_format, dec_format)]



In [5]:
# Here I'm rewriting our matching algorithm using the search_around_sky() function
# It may not always be the best option, but at least for these double peaked catalogs, I think I'm going to run 
# with it

def match_tables_pairs(t1,t2,match_tol):
    if 'level_0' in t1.columns:
        t1.drop(labels=['level_0'], axis=1, inplace=True)
    t1.reset_index(drop=False, inplace=True)
    if 'level_0' in t2.columns:
        t2.drop(labels=['level_0'], axis=1, inplace=True)
    t2.reset_index(inplace=True, drop=False)
    t1['Table_flag'] = 'Table1'
    t2['Table_flag'] = 'Table2'
    # First we begin by matching RA1 and Dec1 of t1 to RA1 and Dec1 of t2
    c1 = SkyCoord(ra=t1['RA1_deg']*u.degree, dec=t1['Dec1_deg']*u.degree) # Storing coordinates for table 1
    c2 = SkyCoord(ra=t2['RA1_deg']*u.degree, dec=t2['Dec1_deg']*u.degree) # storing coordinates for table 2
    # Adding a match tolerance here, with user input for the function
    max_sep = match_tol * u.arcsec # The max match tolerance will be 5''
    #idx2, d2d2, d3d2 = match_coordinates_sky(c1, c2) # Now matching table 1 to table 2
    idx1, idx2, _, _ = c2.search_around_sky(c1, max_sep) 
    # idx1 and idx2 are the indices in table 1 and table 2 which are the closest matching rows to each other
    # Note, we should not need to cross match RA1 vs. RA2, across table because the double peaked sources only have
    # a single set of coordinates at this point
    # We need to make tables for t1 and t2 that do not include the matched items
    t1unique = (t1[~t1['index'].isin(idx1)]).reset_index(drop=True)
    t2unique = (t2[~t2['index'].isin(idx2)]).reset_index(drop=True)
    # And then we need a table for the matches items where we ensure they are properly matching (SDSS names should \
    # be the same), and then remove the duplicates, store the relevant info from the second table, and concatenate \
    # this with the primary table
    tmatches_1 = pd.concat([(t1.iloc[idx1]),(t2.iloc[idx2])]).sort_values(by='Name1').reset_index(drop=True)
    tunique_1 = pd.concat([t1unique, t2unique]).sort_values(by='Name1').reset_index(drop=True)
    #
    # Now we need to match RA1 and Dec1 of t1 to RA2 and Dec2 of t2 in case the pairs are mixeup between tables
    c1 = SkyCoord(ra=t1['RA1_deg']*u.degree, dec=t1['Dec1_deg']*u.degree) # Storing coordinates for table 1
    c2 = SkyCoord(ra=t2['RA2_deg']*u.degree, dec=t2['Dec2_deg']*u.degree) # storing coordinates for table 2
    # Adding a match tolerance here, with user input for the function
    max_sep = match_tol * u.arcsec # The max match tolerance will be 5''
    #idx2, d2d2, d3d2 = match_coordinates_sky(c1, c2) # Now matching table 1 to table 2
    idx1_2, idx2_2, _, _ = c2.search_around_sky(c1, max_sep) 
    # idx1 and idx2 are the indices in table 1 and table 2 which are the closest matching rows to each other
    # Note, we should not need to cross match RA1 vs. RA2, across table because the double peaked sources only have
    # a single set of coordinates at this point
    # We need to make tables for t1 and t2 that do not include the matched items
    t1unique = (t1[~t1['index'].isin(idx1_2)]).reset_index(drop=True)
    t2unique = (t2[~t2['index'].isin(idx2_2)]).reset_index(drop=True)
    # And then we need a table for the matches items where we ensure they are properly matching (SDSS names should \
    # be the same), and then remove the duplicates, store the relevant info from the second table, and concatenate \
    # this with the primary table
    tmatches_2 = pd.concat([(t1.iloc[idx1_2]),(t2.iloc[idx2_2])]).sort_values(by='Name1').reset_index(drop=True)
    tunique_2 = pd.concat([t1unique, t2unique]).sort_values(by='Name1').reset_index(drop=True)
    #
    return tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2


# Here I'm rewriting our matching algorithm using the search_around_sky() function
# It may not always be the best option, but at least for these double peaked catalogs, I think I'm going to run 
# with it

def match_tables_fib(t1,t2,match_tol):
    if 'level_0' in t1.columns:
        t1.drop(labels=['level_0'], axis=1, inplace=True)
    t1.reset_index(drop=False, inplace=True)
    if 'level_0' in t2.columns:
        t2.drop(labels=['level_0'], axis=1, inplace=True)
    t2.reset_index(inplace=True, drop=False)
    t1['Table_flag'] = 'Table1'
    t2['Table_flag'] = 'Table2'
    # First we begin by matching RA1 and Dec1 of t1 to RA1 and Dec1 of t2
    c1 = SkyCoord(ra=t1['RA1_deg']*u.degree, dec=t1['Dec1_deg']*u.degree) # Storing coordinates for table 1
    c2 = SkyCoord(ra=t2['RA1_deg']*u.degree, dec=t2['Dec1_deg']*u.degree) # storing coordinates for table 2
    # Adding a match tolerance here, with user input for the function
    max_sep = match_tol * u.arcsec # The max match tolerance will be 5''
    #idx2, d2d2, d3d2 = match_coordinates_sky(c1, c2) # Now matching table 1 to table 2
    idx1, idx2, _, _ = c2.search_around_sky(c1, max_sep) 
    # idx1 and idx2 are the indices in table 1 and table 2 which are the closest matching rows to each other
    # Note, we should not need to cross match RA1 vs. RA2, across table because the double peaked sources only have
    # a single set of coordinates at this point
    # We need to make tables for t1 and t2 that do not include the matched items
    t1unique = (t1[~t1['index'].isin(idx1)]).reset_index(drop=True)
    t2unique = (t2[~t2['index'].isin(idx2)]).reset_index(drop=True)
    # And then we need a table for the matches items where we ensure they are properly matching (SDSS names should \
    # be the same), and then remove the duplicates, store the relevant info from the second table, and concatenate \
    # this with the primary table
    tmatches = pd.concat([(t1.iloc[idx1]),(t2.iloc[idx2])]).sort_values(by='Name').reset_index(drop=True)
    tunique = pd.concat([t1unique, t2unique]).sort_values(by='Name').reset_index(drop=True)
    #
    #t1matches.loc[t1matches['index'].isin(c1_dups['idx1']), 'Paper(s)'] += " ; " + t2['Paper(s)'][0]
    #t1matches.loc[t1matches['index'].isin(c1_dups['idx1']), 'BibCode(s)'] += " ; " + t2['BibCode(s)'][0]
    #t1matches.loc[t1matches['index'].isin(c1_dups['idx1']), 'DOI(s)'] += " ; " + t2['DOI(s)'][0]
    return tunique, tmatches, idx1, idx2


def match_tables_fib_2RA(t1,t2,match_tol):
    if 'level_0' in t1.columns:
        t1.drop(labels=['level_0'], axis=1, inplace=True)
    t1.reset_index(drop=False, inplace=True)
    if 'level_0' in t2.columns:
        t2.drop(labels=['level_0'], axis=1, inplace=True)
    t2.reset_index(inplace=True, drop=False)
    t1['Table_flag'] = 'Table1'
    t2['Table_flag'] = 'Table2'
    # First we begin by matching RA1 and Dec1 of t1 to RA1 and Dec1 of t2
    c1 = SkyCoord(ra=t1['RA1_deg']*u.degree, dec=t1['Dec1_deg']*u.degree) # Storing coordinates for table 1
    c2 = SkyCoord(ra=t2['RA2_deg']*u.degree, dec=t2['Dec2_deg']*u.degree) # storing coordinates for table 2
    # Adding a match tolerance here, with user input for the function
    max_sep = match_tol * u.arcsec # The max match tolerance will be 5''
    #idx2, d2d2, d3d2 = match_coordinates_sky(c1, c2) # Now matching table 1 to table 2
    idx1, idx2, _, _ = c2.search_around_sky(c1, max_sep) 
    # idx1 and idx2 are the indices in table 1 and table 2 which are the closest matching rows to each other
    # Note, we should not need to cross match RA1 vs. RA2, across table because the double peaked sources only have
    # a single set of coordinates at this point
    # We need to make tables for t1 and t2 that do not include the matched items
    t1unique = (t1[~t1['index'].isin(idx1)]).reset_index(drop=True)
    t2unique = (t2[~t2['index'].isin(idx2)]).reset_index(drop=True)
    # And then we need a table for the matches items where we ensure they are properly matching (SDSS names should \
    # be the same), and then remove the duplicates, store the relevant info from the second table, and concatenate \
    # this with the primary table
    tmatches = pd.concat([(t1.iloc[idx1]),(t2.iloc[idx2])]).sort_values(by='Name').reset_index(drop=True)
    tunique = pd.concat([t1unique, t2unique]).sort_values(by='Name').reset_index(drop=True)
    #
    #t1matches.loc[t1matches['index'].isin(c1_dups['idx1']), 'Paper(s)'] += " ; " + t2['Paper(s)'][0]
    #t1matches.loc[t1matches['index'].isin(c1_dups['idx1']), 'BibCode(s)'] += " ; " + t2['BibCode(s)'][0]
    #t1matches.loc[t1matches['index'].isin(c1_dups['idx1']), 'DOI(s)'] += " ; " + t2['DOI(s)'][0]
    return tunique, tmatches, idx1, idx2




In [6]:
# Inada+2008

# J093207.15+072251.3 --> Redshift in first column 1.993 does not match exactly what is shown in the last column
# --> 1.994
# J100859.55+035104.4 --> redshift in first column 1.746 does not match exactly what is shown in the last column
# --> 1.745

#for table 2 first

# I included all listed pairs, as well as anything that said 'no lensing object' or simply listed a separation

inada2008t2 = pd.read_csv('Tables/Inada2008/Inada2008_t2.csv', sep=',')

# Excluding the no lens objects because they believe these are still likely lenses
#inada2008t2 = inada2008t2[inada2008t2['Comment']!='No lens object']

# Since the format of the catalog will require names for both components, we'll add in a column 'Name2' which for \
# these and similar targets will be duplicates of the first 'Name column'. Same goes for z2, etc. 

inada2008t2['Name1'] = inada2008t2['iSDSS']
inada2008t2['Name2'] = '-99'
#inada2008t2['z2'] = -99
#inada2008t2['z1_type'] = "spec"
#inada2008t2['z2_type'] = "-99"

for index, row in inada2008t2.iterrows():
    if row['z1']<0:
        inada2008t2.at[index, 'z1'] = -99
        inada2008t2.at[index, 'z1_type'] = '-99'
    if row['z2']<0:
        inada2008t2.at[index, 'z2'] = -99
        inada2008t2.at[index, 'z2_type'] = '-99'

# Now converting the naming convention to RA and Dec and adding some informative columns
name_to_coords(inada2008t2,inada2008t2['Name1'])
#name_to_coords2(inada2008t2,inada2008t2['Name2'])

inada2008t2['RA1'] = inada2008t2['RA']
inada2008t2['Dec1'] = inada2008t2['Dec']

## Converting the coordinates
coordconvert = SkyCoord(ra = inada2008t2['RA1'], dec = inada2008t2['Dec1'], frame='icrs', unit = (u.hourangle, u.deg))
inada2008t2['RA1_deg'] = coordconvert.ra.degree
inada2008t2['Dec1_deg'] = coordconvert.dec.degree

inada2008t2['RA2'] = -99
inada2008t2['Dec2'] = -99
inada2008t2['RA2_deg'] = -99
inada2008t2['Dec2_deg'] = -99

# Adding details about the coordinates
inada2008t2['Equinox1'] = "J2000"
inada2008t2['Coordinate_waveband1'] = "Optical"
inada2008t2['Coordinate_Source1'] = "SDSS"

inada2008t2['Equinox2'] = '-99'
inada2008t2['Coordinate_waveband2'] = '-99'
inada2008t2['Coordinate_Source2'] = '-99'

inada2008t2['System Type'] = 'Dual AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
inada2008t2['Brightness1'] = -100
inada2008t2['Brightness_band1'] = -100
inada2008t2['Brightness_type1'] = -100

inada2008t2['Brightness2'] = -100
inada2008t2['Brightness_band2'] = -100
inada2008t2['Brightness_type2'] = -100

inada2008t2['Sep'] = inada2008t2['theta_SDSS']

#inada2008t2['Sep(kpc)'] = inada2008t2['Sep']*((cosmo.arcsec_per_kpc_proper(inada2008t2['z']))**(-1))

inada2008t2['dV'] = -99 #(2.99e+5)*((1+inada2008t2['z1'])**2 - (1+inada2008t2['z2'])**2)/((1+inada2008t2['z1'])**2+(1+inada2008t2['z2'])**2)
# dV will be zero until we include follow-up observations that show separate redshifts

for index, row in inada2008t2.iterrows():
    if (row['z1']!=-99) & (row['z2']!=-99):
        inada2008t2.at[index, 'dV'] = (2.99e+5)*((1+inada2008t2.at[index,'z1'])**2 - (1+inada2008t2.at[index,'z2'])**2)/((1+inada2008t2.at[index,'z1'])**2+(1+inada2008t2.at[index,'z2'])**2)
    else:
        inada2008t2.at[index, 'dV'] = -99

# Adding information about the paper and the selection method
#inada2008t2['Selection Method'] = "-99" #DPSELs
#inada2008t2['Confirmation Method'] = "-99"
inada2008t2['Paper(s)'] = "Inada+2008"
inada2008t2['BibCode(s)'] = "2008AJ....135..496I"
inada2008t2['DOI(s)'] = "https://doi.org/10.1088/0004-6256/135/2/496"

inada2008t2['Notes'] = '-99'

# And dropping any columns that we don't need....
#inada2008t2.drop(labels=['e_z','Del1','e_Del1','Del2','e_Del2','FOIII1','e_FOIII1','FOIII2','e_FOIII2','D'],\
#              axis=1, inplace=True)

#inada2008t2



In [7]:
# Inada+2008

# J093207.15+072251.3 --> Redshift in first column 1.993 does not match exactly what is shown in the last column
# --> 1.994
# J100859.55+035104.4 --> redshift in first column 1.746 does not match exactly what is shown in the last column
# --> 1.745

# This set of objects is excluded since it presumably makes up a triple image:
#SDSSJ161953.24+351321.8,1.901,18.64,,,
#SDSSJ161953.45+351323.5,,19.50, 3.13,
#No lensing object,SDSSJ161952.82+351315.4,,19.89, 8.20,No lensing object,

# All inconclusive cases are included; note they don't use 'inconclusive', that's me using that term here
# originally missed one object at the end, but it's not included (30 Sept 2023)

inada2008t3 = pd.read_csv('Tables/Inada2008/Inada2008_t3.csv', sep=',')

# Excluding the no lens objects because they believe these are still likely lenses
#inada2008t3 = inada2008t3[inada2008t3['Comment']!='No lens object']

# Since the format of the catalog will require names for both components, we'll add in a column 'Name2' which for \
# these and similar targets will be duplicates of the first 'Name column'. Same goes for z2, etc. 

#inada2008t3['z1_type'] = "spec"
#inada2008t3['z2_type'] = "spec"

for index, row in inada2008t3.iterrows():
    if row['z1']<0:
        inada2008t3.at[index, 'z1'] = -99
        inada2008t3.at[index, 'z1_type'] = '-99'
    if row['z2']<0:
        inada2008t3.at[index, 'z2'] = -99
        inada2008t3.at[index, 'z2_type'] = '-99'


# Now converting the naming convention to RA and Dec and adding some informative columns
name_to_coords1(inada2008t3,inada2008t3['Name1'])
name_to_coords2(inada2008t3,inada2008t3['Name2'])

## Converting the coordinates
coordconvert = SkyCoord(ra = inada2008t3['RA1'], dec = inada2008t3['Dec1'], frame='icrs', unit = (u.hourangle, u.deg))
inada2008t3['RA1_deg'] = coordconvert.ra.degree
inada2008t3['Dec1_deg'] = coordconvert.dec.degree

coordconvert = SkyCoord(ra = inada2008t3['RA2'], dec = inada2008t3['Dec2'], frame='icrs', unit = (u.hourangle, u.deg))
inada2008t3['RA2_deg'] = coordconvert.ra.degree
inada2008t3['Dec2_deg'] = coordconvert.dec.degree

# Adding details about the coordinates
inada2008t3['Equinox1'] = "J2000"
inada2008t3['Coordinate_waveband1'] = "Optical"
inada2008t3['Coordinate_Source1'] = "SDSS"

inada2008t3['Equinox2'] = "J2000"
inada2008t3['Coordinate_waveband2'] = "Optical"
inada2008t3['Coordinate_Source2'] = "SDSS"

inada2008t3['System Type'] = 'Dual AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
inada2008t3['Brightness1'] = -100
inada2008t3['Brightness_band1'] = -100
inada2008t3['Brightness_type1'] = -100

inada2008t3['Brightness2'] = -100
inada2008t3['Brightness_band2'] = -100
inada2008t3['Brightness_type2'] = -100

inada2008t3['Sep'] = inada2008t3['thetaSDSS2']

#inada2008t3['Sep(kpc)'] = inada2008t3['Sep']*((cosmo.arcsec_per_kpc_proper(inada2008t3['z']))**(-1))

inada2008t3['dV'] = -99 #(2.99e+5)*((1+inada2008t3['z1'])**2 - (1+inada2008t3['z2'])**2)/((1+inada2008t3['z1'])**2+(1+inada2008t3['z2'])**2)
# dV will be zero until we include follow-up observations that show separate redshifts

for index, row in inada2008t3.iterrows():
    if (row['z1']!=-99) & (row['z2']!=-99):
        inada2008t3.at[index, 'dV'] = (2.99e+5)*((1+inada2008t3.at[index,'z1'])**2 - (1+inada2008t3.at[index,'z2'])**2)/((1+inada2008t3.at[index,'z1'])**2+(1+inada2008t3.at[index,'z2'])**2)
    else:
        inada2008t3.at[index, 'dV'] = -99

# Adding information about the paper and the selection method
#inada2008t3['Selection Method'] = "-99" #DPSELs
#inada2008t3['Confirmation Method'] = "-99"
inada2008t3['Paper(s)'] = "Inada+2008"
inada2008t3['BibCode(s)'] = "2008AJ....135..496I"
inada2008t3['DOI(s)'] = "https://doi.org/10.1088/0004-6256/135/2/496"

inada2008t3['Notes'] = '-99'

# And dropping any columns that we don't need....
#inada2008t3.drop(labels=['e_z','Del1','e_Del1','Del2','e_Del2','FOIII1','e_FOIII1','FOIII2','e_FOIII2','D'],\
#              axis=1, inplace=True)

#inada2008t3


In [8]:
inada2008 = pd.concat([inada2008t2,inada2008t3])

#inada2008

In [9]:
# Inada+2010
#for table 2 first

# No inconclusive cases missed here. They don't use 'inconclusive'; but I've included all cases of 'no lens'


inada2010t2 = pd.read_csv('Tables/inada2010/inada2010_t2.csv', sep=',')

# Excluding the no lens objects because they believe these are still likely lenses
#inada2010t2 = inada2010t2[inada2010t2['Comment']!='No lens object']

# Since the format of the catalog will require names for both components, we'll add in a column 'Name2' which for \
# these and similar targets will be duplicates of the first 'Name column'. Same goes for z2, etc. 

inada2010t2['Name1'] = inada2010t2['Name']
inada2010t2['Name2'] = '-99'
#inada2010t2['z2'] = -99
inada2010t2['z1_type'] = "spec"
inada2010t2['z2_type'] = "spec"

name_to_coords(inada2010t2,inada2010t2['Name'])

inada2010t2['RA1'] = inada2010t2['RA'] 
inada2010t2['Dec1'] = inada2010t2['Dec']

# Converting the coordinates
coordconvert = SkyCoord(ra = inada2010t2['RA'], dec = inada2010t2['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
inada2010t2['RA1_deg'] = coordconvert.ra.degree
inada2010t2['Dec1_deg'] = coordconvert.dec.degree

# Adding in a second set of coordinates for the 'secondary'
inada2010t2['RA2'] = -99
inada2010t2['Dec2'] = -99

inada2010t2['RA2_deg'] = -99
inada2010t2['Dec2_deg'] = -99


# Adding details about the coordinates
inada2010t2['Equinox1'] = "J2000"
inada2010t2['Coordinate_waveband1'] = "Optical"
inada2010t2['Coordinate_Source1'] = "SDSS"

inada2010t2['Equinox2'] = '-99'
inada2010t2['Coordinate_waveband2'] = '-99'
inada2010t2['Coordinate_Source2'] = '-99'

inada2010t2['System Type'] = 'Dual AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
inada2010t2['Brightness1'] = -100
inada2010t2['Brightness_band1'] = -100
inada2010t2['Brightness_type1'] = -100

inada2010t2['Brightness2'] = -100
inada2010t2['Brightness_band2'] = -100
inada2010t2['Brightness_type2'] = -100

inada2010t2['Sep'] = inada2010t2['sep_as']

#inada2010t2['Sep(kpc)'] = inada2010t2['Sep']*((cosmo.arcsec_per_kpc_proper(inada2010t2['z']))**(-1))

inada2010t2['dV'] = -99
# dV will be zero until we include follow-up observations that show separate redshifts

for index, row in inada2010t2.iterrows():
    if row['z1']<0:
        inada2010t2.at[index, 'z1'] = -99
        inada2010t2.at[index, 'z1_type'] = '-99'
    if row['z2']<0:
        inada2010t2.at[index, 'z2'] = -99
        inada2010t2.at[index, 'z2_type'] = '-99'

for index, row in inada2010t2.iterrows():
    if (row['z1']!=-99) & (row['z2']!=-99):
        inada2010t2.at[index, 'dV'] = (2.99e+5)*((1+inada2010t2.at[index,'z1'])**2 - (1+inada2010t2.at[index,'z2'])**2)/((1+inada2010t2.at[index,'z1'])**2+(1+inada2010t2.at[index,'z2'])**2)
    else:
        inada2010t2.at[index, 'dV'] = -99

# Adding information about the paper and the selection method
#inada2010t2['Selection Method'] = "-99" #DPSELs
#inada2010t2['Confirmation Method'] = "-99"
inada2010t2['Paper(s)'] = "Inada+2010"
inada2010t2['BibCode(s)'] = "2010AJ....140..403I"
inada2010t2['DOI(s)'] = "https://doi.org/10.1088/0004-6256/140/2/403"

inada2010t2['Notes'] = '-99'

# And dropping any columns that we don't need....
#inada2010t2.drop(labels=['e_z','Del1','e_Del1','Del2','e_Del2','FOIII1','e_FOIII1','FOIII2','e_FOIII2','D'],\
#              axis=1, inplace=True)

#inada2010t2


In [10]:
# Inada+2010
#for table 3

# This was nicely formatted already. I removed all obvious projected pairs, but I've left a few pairs that \
# we can remove here, especially the ones that Hennawi already found (those are marked... we probably will just \
# throw those out....)


inada2010t3 = pd.read_csv('Tables/inada2010/inada2010_t3.csv', sep=',')

# Excluding the no lens objects because they believe these are still likely lenses
#inada2010t3 = inada2010t3[inada2010t3['Comment']!='No lens object']
#inada2010t3 = inada2010t3[inada2010t3['Comment']!='no lens object']
#inada2010t3 = inada2010t3[inada2010t3['z2']>0]

# Since the format of the catalog will require names for both components, we'll add in a column 'Name2' which for \
# these and similar targets will be duplicates of the first 'Name column'. Same goes for z2, etc. 

inada2010t3['z1_type'] = "spec"
inada2010t3['z2_type'] = "spec"

# Now converting the naming convention to RA and Dec and adding some informative columns
name_to_coords1(inada2010t3,inada2010t3['Name1'])
name_to_coords2(inada2010t3,inada2010t3['Name2'])

## Converting the coordinates
coordconvert = SkyCoord(ra = inada2010t3['RA1'], dec = inada2010t3['Dec1'], frame='icrs', unit = (u.hourangle, u.deg))
inada2010t3['RA1_deg'] = coordconvert.ra.degree
inada2010t3['Dec1_deg'] = coordconvert.dec.degree

coordconvert = SkyCoord(ra = inada2010t3['RA2'], dec = inada2010t3['Dec2'], frame='icrs', unit = (u.hourangle, u.deg))
inada2010t3['RA2_deg'] = coordconvert.ra.degree
inada2010t3['Dec2_deg'] = coordconvert.dec.degree

# Adding details about the coordinates
inada2010t3['Equinox1'] = "J2000"
inada2010t3['Coordinate_waveband1'] = "Optical"
inada2010t3['Coordinate_Source1'] = "SDSS"

inada2010t3['Equinox2'] = "J2000"
inada2010t3['Coordinate_waveband2'] = "Optical"
inada2010t3['Coordinate_Source2'] = "SDSS"

inada2010t3['System Type'] = 'Dual AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
inada2010t3['Brightness1'] = -100
inada2010t3['Brightness_band1'] = -100
inada2010t3['Brightness_type1'] = -100

inada2010t3['Brightness2'] = -100
inada2010t3['Brightness_band2'] = -100
inada2010t3['Brightness_type2'] = -100

inada2010t3['Sep'] = inada2010t3['Sep_as']

#inada2010t3['Sep(kpc)'] = inada2010t3['Sep']*((cosmo.arcsec_per_kpc_proper(inada2010t3['z']))**(-1))

inada2010t3['dV'] = (2.99e+5)*((1+inada2010t3['z1'])**2 - (1+inada2010t3['z2'])**2)/((1+inada2010t3['z1'])**2+(1+inada2010t3['z2'])**2)

# Adding information about the paper and the selection method
#inada2010t3['Selection Method'] = "-99" #DPSELs
#inada2010t3['Confirmation Method'] = "-99"
inada2010t3['Paper(s)'] = "Inada+2010"
inada2010t3['BibCode(s)'] = "2010AJ....140..403I"
inada2010t3['DOI(s)'] = "https://doi.org/10.1088/0004-6256/140/2/403"

inada2010t3['Notes'] = '-99'

# I fucked myself over here... I'm going to have to back and thoroughly rematch again because I wasn't being \
# careful here. Fuck. 
inada2010t3_forlater = inada2010t3[np.abs(inada2010t3['dV'])>10000]
inada2010t3 = inada2010t3[np.abs(inada2010t3['dV'])<10000]
# dV will be zero until we include follow-up observations that show separate redshifts


# And dropping any columns that we don't need....
#inada2010t3.drop(labels=['e_z','Del1','e_Del1','Del2','e_Del2','FOIII1','e_FOIII1','FOIII2','e_FOIII2','D'],\
#              axis=1, inplace=True)

#inada2010t3


In [11]:
# here we are going to save this tabel for later. We will need to match this against the full MAC in the match \
# catalogs notebook. Otherwise it will be too much work to match here, then match alll the way through the other \
# notebook

for index, row in inada2010t3_forlater.iterrows():
    if row['z1']==0:
        inada2010t3_forlater.at[index, 'z1'] = -99
        inada2010t3_forlater.at[index, 'z1_type'] = '-99'
    if row['z2']==0:
        inada2010t3_forlater.at[index, 'z2'] = -99
        inada2010t3_forlater.at[index, 'z2_type'] = '-99'


for index, row in inada2010t3_forlater.iterrows():
    if (row['z1']!=-99) & (row['z2']!=-99):
        inada2010t3_forlater.at[index, 'dV'] = (2.99e+5)*((1+inada2010t3_forlater.at[index,'z1'])**2 - (1+inada2010t3_forlater.at[index,'z2'])**2)/((1+inada2010t3_forlater.at[index,'z1'])**2+(1+inada2010t3_forlater.at[index,'z2'])**2)
    else:
        inada2010t3_forlater.at[index, 'dV'] = -99


inada2010t3_forlater.to_csv('inada2010t3_forlatermatching.csv', sep=',', index=False)

#inada2010t3_forlater


In [12]:
#inada2010t3_forlater

In [13]:
inada2010 = pd.concat([inada2010t2,inada2010t3])

#inada2010

In [14]:
# Matching the Inada 2008 and 2010 tables here

tunique, tmatches, idx1, idx2 = match_tables_fib(inada2008,inada2010,5)

#print(len(tmatches)) there are no matches!

the_whills = pd.concat([inada2008,inada2010])

# verified

In [15]:
# Inada+2012

# No inconclusive cases to include here. All 'no lens' cases have been included.
# I verified that I'm not missing any possible pairs from that table

inada2012t3 = pd.read_csv('Tables/Inada2012/inada2012_t3.csv', sep=',')

#inada2012t3 = inada2012t3[inada2012t3['Com']!='No lens object']
#inada2012t3 = inada2012t3[inada2012t3['Com']!='no lens object']
#inada2012t3 = inada2012t3[inada2012t3['Com']!='separation 0.7 arcsec']
#inada2012t3 = inada2012t3[inada2012t3['z2']>0]

inada2012t3['Name1'] = inada2012t3['Name']
inada2012t3['Name2'] = '-99'
inada2012t3['z1_type'] = "spec"
inada2012t3['z2_type'] = "spec"

for index, row in inada2012t3.iterrows():
    if row['z1']<0:
        inada2012t3.at[index, 'z1'] = -99
        inada2012t3.at[index, 'z1_type'] = '-99'
    if row['z2']<0:
        inada2012t3.at[index, 'z2'] = -99
        inada2012t3.at[index, 'z2_type'] = '-99'

# Now converting the naming convention to RA and Dec and adding some informative columns
name_to_coords(inada2012t3,inada2012t3['Name'])

inada2012t3['RA1'] = inada2012t3['RA']
inada2012t3['Dec1'] = inada2012t3['Dec']
# Converting the coordinates
coordconvert = SkyCoord(ra = inada2012t3['RA'], dec = inada2012t3['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
inada2012t3['RA1_deg'] = coordconvert.ra.degree
inada2012t3['Dec1_deg'] = coordconvert.dec.degree

# Adding in a second set of coordinates for the 'secondary'
inada2012t3['RA2'] = -99
inada2012t3['Dec2'] = -99

inada2012t3['RA2_deg'] = -99
inada2012t3['Dec2_deg'] = -99

# Adding details about the coordinates
inada2012t3['Equinox1'] = "J2000"
inada2012t3['Coordinate_waveband1'] = "Optical"
inada2012t3['Coordinate_Source1'] = "SDSS"

inada2012t3['Equinox2'] = '-99'
inada2012t3['Coordinate_waveband2'] = '-99'
inada2012t3['Coordinate_Source2'] = '-99'

inada2012t3['System Type'] = 'Dual AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
inada2012t3['Brightness1'] = -100
inada2012t3['Brightness_band1'] = -100
inada2012t3['Brightness_type1'] = -100

inada2012t3['Brightness2'] = -100
inada2012t3['Brightness_band2'] = -100
inada2012t3['Brightness_type2'] = -100

inada2012t3['Sep'] = inada2012t3['theta']

#inada2012t3['Sep(kpc)'] = inada2012t3['Sep']*((cosmo.arcsec_per_kpc_proper(inada2012t3['z']))**(-1))

inada2012t3['dV'] = -99
for index, row in inada2012t3.iterrows():
    if (row['z1']!=-99) & (row['z2']!=-99):
        inada2012t3.at[index, 'dV'] = (2.99e+5)*((1+inada2012t3.at[index,'z1'])**2 - (1+inada2012t3.at[index,'z2'])**2)/((1+inada2012t3.at[index,'z1'])**2+(1+inada2012t3.at[index,'z2'])**2)
    else:
        inada2012t3.at[index, 'dV'] = -99
# dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
#inada2012t3['Selection Method'] = "-99" #DPSELs
#inada2012t3['Confirmation Method'] = "-99"
inada2012t3['Paper(s)'] = "Inada+2012"
inada2012t3['BibCode(s)'] = "2012AJ....143..119I"
inada2012t3['DOI(s)'] = "https://doi.org/10.1088/0004-6256/143/5/119"

inada2012t3['Notes'] = '-99'

# And dropping any columns that we don't need....
#inada2012t3.drop(labels=['e_z','Del1','e_Del1','Del2','e_Del2','FOIII1','e_FOIII1','FOIII2','e_FOIII2','D'],\
#              axis=1, inplace=True)

#inada2012t3


In [16]:
# Inada+2012

# All objects that are not QSO+stars or QSOs+nonQSOs are included, except for this apparent triple image:
#SDSSJ162033.71+121121.2,1.629,18.66,SDSSJ162032.65+121112.8,0,18.33,17.90,no lensing object
#SDSSJ162033.71+121121.2,1.629,18.66,SDSSJ162033.47+121117.8,0,19.53,5.11,no lensing object

inada2012t4 = pd.read_csv('Tables/Inada2012/inada2012_t4.csv', sep=',')

#inada2012t4 = inada2012t4[inada2012t4['Com']!='No lens object']
#inada2012t4 = inada2012t4[inada2012t4['Com']!='no lens object']
#inada2012t4 = inada2012t4[inada2012t4['z2']>0]

#inada2012t4['Name1'] = inada2012t4['Name']
inada2012t4['z1_type'] = "spec"
inada2012t4['z2_type'] = "spec"

# Now converting the naming convention to RA and Dec and adding some informative columns
name_to_coords1(inada2012t4,inada2012t4['Name1'])
name_to_coords2(inada2012t4,inada2012t4['Name2'])

# Converting the coordinates
coordconvert = SkyCoord(ra = inada2012t4['RA1'], dec = inada2012t4['Dec1'], frame='icrs', unit = (u.hourangle, u.deg))
inada2012t4['RA1_deg'] = coordconvert.ra.degree
inada2012t4['Dec1_deg'] = coordconvert.dec.degree

coordconvert = SkyCoord(ra = inada2012t4['RA2'], dec = inada2012t4['Dec2'], frame='icrs', unit = (u.hourangle, u.deg))
inada2012t4['RA2_deg'] = coordconvert.ra.degree
inada2012t4['Dec2_deg'] = coordconvert.dec.degree

# Adding details about the coordinates
inada2012t4['Equinox1'] = "J2000"
inada2012t4['Coordinate_waveband1'] = "Optical"
inada2012t4['Coordinate_Source1'] = "SDSS"

inada2012t4['Equinox2'] = "J2000"
inada2012t4['Coordinate_waveband2'] = "Optical"
inada2012t4['Coordinate_Source2'] = "SDSS"

inada2012t4['System Type'] = 'Dual AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
inada2012t4['Brightness1'] = -100
inada2012t4['Brightness_band1'] = -100
inada2012t4['Brightness_type1'] = -100

inada2012t4['Brightness2'] = -100
inada2012t4['Brightness_band2'] = -100
inada2012t4['Brightness_type2'] = -100

inada2012t4['Sep'] = inada2012t4['theta']

#inada2012t4['Sep(kpc)'] = inada2012t4['Sep']*((cosmo.arcsec_per_kpc_proper(inada2012t4['z']))**(-1))

# Adding information about the paper and the selection method
#inada2012t4['Selection Method'] = "-99" #DPSELs
#inada2012t4['Confirmation Method'] = "-99"
inada2012t4['Paper(s)'] = "Inada+2012"
inada2012t4['BibCode(s)'] = "2012AJ....143..119I"
inada2012t4['DOI(s)'] = "https://doi.org/10.1088/0004-6256/143/5/119"

inada2012t4['Notes'] = '-99'

inada2012t4['dV'] = (2.99e+5)*((1+inada2012t4['z1'])**2 - (1+inada2012t4['z2'])**2)/((1+inada2012t4['z1'])**2+(1+inada2012t4['z2'])**2)
inada2012t4_forlater = inada2012t4[np.abs(inada2012t4['dV'])>3000]
inada2012t4 = inada2012t4[np.abs(inada2012t4['dV'])<3000]
# dV will be zero until we include follow-up observations that show separate redshifts


# And dropping any columns that we don't need....
#inada2012t4.drop(labels=['e_z','Del1','e_Del1','Del2','e_Del2','FOIII1','e_FOIII1','FOIII2','e_FOIII2','D'],\
#              axis=1, inplace=True)

#inada2012t4


In [17]:
#inada2012t4_forlater

In [18]:
# saving the 'lost' objects due to the stupid dV cut above. We will need to match this against the MAC in \
# the matching notebook

for index, row in inada2012t4_forlater.iterrows():
    if row['z1']==0:
        inada2012t4_forlater.at[index, 'z1'] = -99
        inada2012t4_forlater.at[index, 'z1_type'] = '-99'
    if row['z2']==0:
        inada2012t4_forlater.at[index, 'z2'] = -99
        inada2012t4_forlater.at[index, 'z2_type'] = '-99'


for index, row in inada2012t4_forlater.iterrows():
    if (row['z1']!=-99) & (row['z2']!=-99):
        inada2012t4_forlater.at[index, 'dV'] = (2.99e+5)*((1+inada2012t4_forlater.at[index,'z1'])**2 - (1+inada2012t4_forlater.at[index,'z2'])**2)/((1+inada2012t4_forlater.at[index,'z1'])**2+(1+inada2012t4_forlater.at[index,'z2'])**2)
    else:
        inada2012t4_forlater.at[index, 'dV'] = -99


inada2012t4_forlater.to_csv('inada2012t4_forlatermatching.csv', sep=',', index=False)

#inada2012t4_forlater


In [19]:
inada2012 = pd.concat([inada2012t3,inada2012t4])

#inada2012

In [20]:
# adding in Inada 2012 here
tunique, tmatches, idx1, idx2 = match_tables_fib(inada2008,inada2010,5)

#print(len(tmatches)) #there are no matches!

the_whills = pd.concat([the_whills,inada2012])

# verified


In [21]:
#the_whills

In [22]:
# Here we're adding in the information from Rusu+2013
# SDSS J132059.17+164402.59 and SDSS J132059.73+164405.6 (SDSS J1320+1644)

#rusu2013 = ['SDSSJ132059.17+164402.59','SDSSJ132059.73+164405.6'] # 
#for index, row in the_whills.iterrows():
#    if row['Name1'] in rusu2013:
#        the_whills.at[index, 'Paper(s)'] += ' ; Rusu+2013 '
#        the_whills.at[index, 'BibCode(s)'] += ' ; 2013ApJ...765..139R' 
#        the_whills.at[index, 'DOI(s)'] += ' ; https://doi.org/10.1088/0004-637X/765/2/139'
#        the_whills.at[index, 'Notes']=' Rusu+2013 find this system comprises two QSO images and two galaxies between in a cross-like configuration. Rusu+ argue that this is most likely a lens rather than a QSO pair. All observed differences between the spectra can be attributed to a combination of extinction, microlensing,and intrinsic variability. They cannot indisputably rule out the binary hypothesis however.'
#        the_whills.at[index, 'System Type'] = 'Lens / Binary Quasar'

# Actually, since this isn't in the tables, I'll add this as an individual object



In [23]:
#the_whills

In [24]:
# More+2016 here

#Table 1. Summary of spectroscopic observations. The ‘Selection’ column indicates the method(s) with ‘M’ for morphological selection, ‘C’ for colour
#selection, and ‘S’ for spectroscopic selection (see the text for details). Several candidates were selected by multiple methods. ‘Observation’ indicates the
#telescope (S-Subaru or K-Keck), the date and setup of the spectroscopic observations (see also Section 3.1). ‘Result’ indicates the conclusion from the
#follow-up spectroscopy as well as imaging observations described in Section 3.2. The zQSO are taken from the BOSS DR12 catalogue which has redshifts
#corrected after visual inspection of the spectra. The last column has redshifts from the follow-up spectra except when marked witha which are taken from the
#BOSS spectrum.

#Table 7. Summary of confirmed quasar pairs. See Table 1 for the RA and Dec. of the brighter quasar image (A). The
#positions and the i-band magnitudes in this table are from the SDSS data base except for J0818+0601 where we used the
#astrometrically calibrated SOAR images.

# this table includes quasar pairs that have velocity differences less than 2000 km s^-1, pairs that don't have \
# two spec-z's but are listed as having sdifferent SEDs, and two inconclusive pairs


more2016 = pd.read_csv('Tables/more2016/more2016.csv', sep=',')

# No distinct coordinates are provided. We need to reach out and request them.
# Separations provided for only some targets

# Since the format of the catalog will require names for both components, we'll add in a column 'Name2' which for \
# these and similar targets will be duplicates of the first 'Name column'. Same goes for z2, etc. 

#more2016['Name2'] = more2016['Name']
#more2016['z2'] = more2016['z']
more2016['z1_type'] = "spec"
more2016['z2_type'] = "spec"

for index, row in more2016.iterrows():
    if row['z1']<0:
        more2016.at[index, 'z1'] = -99
        more2016.at[index, 'z1_type'] = '-99'
    if row['z2']<0:
        more2016.at[index, 'z2'] = -99
        more2016.at[index, 'z2_type'] = '-99'

# Now converting the naming convention to RA and Dec and adding some informative columns
#name_to_coords(more2016,more2016['Name'])

## Converting the coordinates
coordconvert = SkyCoord(ra = more2016['RA1'], dec = more2016['Dec1'], frame='icrs', unit = (u.hourangle, u.deg))
more2016['RA1_deg'] = coordconvert.ra.degree
more2016['Dec1_deg'] = coordconvert.dec.degree

## Converting the coordinates
# had to use chatgpt to debug this bug here
more2016['RA2_deg'] = -99
more2016['Dec2_deg'] = -99

# Iterate over rows of the DataFrame
for idx, row in more2016.iterrows():
    try:
        coordconvert = SkyCoord(ra=row['RA2'], dec=row['Dec2'], frame='icrs', unit=(u.hourangle, u.deg))
        more2016.at[idx, 'RA2_deg'] = coordconvert.ra.degree
        more2016.at[idx, 'Dec2_deg'] = coordconvert.dec.degree
    except:
        # If there's an exception, just set this row's values to '-99'
        more2016.at[idx, 'RA2_deg'] = -99
        more2016.at[idx, 'Dec2_deg'] = -99

# Adding details about the coordinates
more2016['Equinox1'] = "J2000"
more2016['Coordinate_waveband1'] = "Optical"
more2016['Coordinate_Source1'] = "SDSS"

more2016['Equinox2'] = "J2000"
more2016['Coordinate_waveband2'] = "Optical"
more2016['Coordinate_Source2'] = "SDSS"

for index, row in more2016.iterrows():
    if row['RA2_deg']<0:
        more2016.at[index, 'Equinox2'] = '-99'
        more2016.at[index, 'Coordinate_waveband2'] = '-99'
        more2016.at[index, 'Coordinate_Source2'] = '-99'

more2016['System Type'] = 'Dual AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
more2016['Brightness1'] = -100
more2016['Brightness_band1'] = -100
more2016['Brightness_type1'] = -100

more2016['Brightness2'] = -100
more2016['Brightness_band2'] = -100
more2016['Brightness_type2'] = -100

more2016['Sep'] = more2016['Sep_as']

#more2016['Sep(kpc)'] = more2016['Sep']*((cosmo.arcsec_per_kpc_proper(more2016['z']))**(-1))

more2016['dV'] = -99
for index, row in more2016.iterrows():
    if (row['z1']!=-99) & (row['z2']!=-99):
        more2016.at[index, 'dV'] = (2.99e+5)*((1+more2016.at[index,'z1'])**2 - (1+more2016.at[index,'z2'])**2)/((1+more2016.at[index,'z1'])**2+(1+more2016.at[index,'z2'])**2)
    else:
        more2016.at[index, 'dV'] = -99
# dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
#more2016['Selection Method'] = "-99" #DPSELs
#more2016['Confirmation Method'] = "-99"
more2016['Paper(s)'] = "More+2016"
more2016['BibCode(s)'] = "2016MNRAS.456.1595M"
more2016['DOI(s)'] = "https://doi.org/10.1093/mnras/stv2813"
more2016['Notes'] = '-99'

# And dropping any columns that we don't need....
#more2016.drop(labels=['e_z','Del1','e_Del1','Del2','e_Del2','FOIII1','e_FOIII1','FOIII2','e_FOIII2','D'],\
#              axis=1, inplace=True)

#more2016



In [25]:
# add in More2016 now...

tunique, tmatches, idx1, idx2 = match_tables_fib(the_whills,more2016,5)

#print(len(tmatches)) #there are no matches!

the_whills = pd.concat([the_whills,more2016])

# verified



In [26]:
# Eftekharzadeh+2017 here
# Sandrinelli+2018 has some notes on an object selected in this sample

#Table 1. Summary of spectroscopic observations. The ‘Selection’ column indicates the method(s) with ‘M’ for morphological selection, ‘C’ for colour
#selection, and ‘S’ for spectroscopic selection (see the text for details). Several candidates were selected by multiple methods. ‘Observation’ indicates the
#telescope (S-Subaru or K-Keck), the date and setup of the spectroscopic observations (see also Section 3.1). ‘Result’ indicates the conclusion from the
#follow-up spectroscopy as well as imaging observations described in Section 3.2. The zQSO are taken from the BOSS DR12 catalogue which has redshifts
#corrected after visual inspection of the spectra. The last column has redshifts from the follow-up spectra except when marked witha which are taken from the
#BOSS spectrum.

#Table 7. Summary of confirmed quasar pairs. See Table 1 for the RA and Dec. of the brighter quasar image (A). The
#positions and the i-band magnitudes in this table are from the SDSS data base except for J0818+0601 where we used the
#astrometrically calibrated SOAR images.


# I'm including 3 objects flagged as a lens but not classified as such (flag 4 but without a class Q lens)
# All quasar pairs (72)
# Alll ambiguous cases (flag 1); 45 of these not including any star+quasars

eftek2017 = pd.read_csv('Tables/Eftekharzadeh2017/Eftekharzadeh2017.csv', sep=',')

eftek2017['z1_type'] = "spec"
eftek2017['z2_type'] = "spec"

eftek2017['RA1_deg'] = eftek2017['RAdeg1']
eftek2017['Dec1_deg'] = eftek2017['DEdeg1']

## Convert Ra and Dec from degrees to sexagesimal format
coords = SkyCoord(ra=eftek2017['RA1_deg']*u.degree, dec=eftek2017['Dec1_deg']*u.degree, frame='icrs')
eftek2017['RA1'] = coords.ra.to_string(u.hour, sep=':', precision=2)
eftek2017['Dec1'] = coords.dec.to_string(u.deg, sep=':', precision=2)

# Convert RA and Dec to the desired format
ra_format = coords.ra.to_string(unit=u.hour, sep='', precision=2, pad=True)
dec_format = coords.dec.to_string(unit=u.deg, sep='', precision=2, alwayssign=True, pad=True)

# Concatenate to form J_format using a loop
eftek2017['Name1'] = ["J" + ra + dec for ra, dec in zip(ra_format, dec_format)]

# Next object
eftek2017['RA2_deg'] = eftek2017['RAdeg2']
eftek2017['Dec2_deg'] = eftek2017['DEdeg2']

## Convert Ra and Dec from degrees to sexagesimal format
coords = SkyCoord(ra=eftek2017['RA2_deg']*u.degree, dec=eftek2017['Dec2_deg']*u.degree, frame='icrs')
eftek2017['RA2'] = coords.ra.to_string(u.hour, sep=':', precision=2)
eftek2017['Dec2'] = coords.dec.to_string(u.deg, sep=':', precision=2)

# Convert RA and Dec to the desired format
ra_format = coords.ra.to_string(unit=u.hour, sep='', precision=2, pad=True)
dec_format = coords.dec.to_string(unit=u.deg, sep='', precision=2, alwayssign=True, pad=True)

# Concatenate to form J_format using a loop
eftek2017['Name2'] = ["J" + ra + dec for ra, dec in zip(ra_format, dec_format)]

for index, row in eftek2017.iterrows():
    if row['z1']<0:
        eftek2017.at[index, 'z1'] = -99
        eftek2017.at[index, 'z1_type'] = '-99'
    if row['z2']<0:
        eftek2017.at[index, 'z2'] = -99
        eftek2017.at[index, 'z2_type'] = '-99'

# Adding details about the coordinates
eftek2017['Equinox1'] = "J2000"
eftek2017['Coordinate_waveband1'] = "Optical"
eftek2017['Coordinate_Source1'] = "SDSS"

eftek2017['Equinox2'] = "J2000"
eftek2017['Coordinate_waveband2'] = "Optical"
eftek2017['Coordinate_Source2'] = "SDSS"

eftek2017['System Type'] = 'Dual AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
eftek2017['Brightness1'] = -100
eftek2017['Brightness_band1'] = -100
eftek2017['Brightness_type1'] = -100

eftek2017['Brightness2'] = -100
eftek2017['Brightness_band2'] = -100
eftek2017['Brightness_type2'] = -100

eftek2017['Sep'] = eftek2017['Sep_as1']

#eftek2017['Sep(kpc)'] = eftek2017['Sep']*((cosmo.arcsec_per_kpc_proper(eftek2017['z']))**(-1))

eftek2017['dV'] = -99
for index, row in eftek2017.iterrows():
    if (row['z1']!=-99) & (row['z2']!=-99):
        eftek2017.at[index, 'dV'] = (2.99e+5)*((1+eftek2017.at[index,'z1'])**2 - (1+eftek2017.at[index,'z2'])**2)/((1+eftek2017.at[index,'z1'])**2+(1+eftek2017.at[index,'z2'])**2)
    else:
        eftek2017.at[index, 'dV'] = -99
        
# dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
#eftek2017['Selection Method'] = "-99" #DPSELs
#eftek2017['Confirmation Method'] = "-99"
eftek2017['Paper(s)'] = "Eftekharzadeh+2017"
eftek2017['BibCode(s)'] = "2017MNRAS.468...77E"
eftek2017['DOI(s)'] = "https://doi.org/10.1093/mnras/stx412"
eftek2017['Notes'] = '-99'

# And dropping any columns that we don't need....
#eftek2017.drop(labels=['e_z','Del1','e_Del1','Del2','e_Del2','FOIII1','e_FOIII1','FOIII2','e_FOIII2','D'],\
#              axis=1, inplace=True)

#eftek2017



In [27]:
# add in eftek2017 now...

tunique, tmatches, idx1, idx2 = match_tables_fib(the_whills,eftek2017,2)

#print(len(tmatches)) #there are no matches!

for i, j in zip(idx1, idx2):
    #print("i:", i, "j:", j)
    #print("Sep:", mcgurk2015t4.at[j, 'NIRC2sep(as)'])
    #print("dV:", mcgurk2015t4.at[j, 'dV[OIII]'])
    the_whills.at[i, 'Sep'] = eftek2017.at[j, 'Sep']
    the_whills.at[i, 'Paper(s)'] += ' ; Eftekharzadeh+2017'
    the_whills.at[i, 'BibCode(s)'] += ' ; 2017MNRAS.468...77E' 
    the_whills.at[i, 'DOI(s)'] += ' ; https://doi.org/10.1093/mnras/stx412'
    the_whills.at[i, 'z2'] = eftek2017.at[j, 'z2']
    #the_whills.at[i, 'Notes'] += ' Fu+2011 companions within 3 arcseconds,'

# dropping the matching indices from the eftek2017 table
eftek2017.drop(idx2, axis=0, inplace=True)
eftek2017.reset_index(drop=True, inplace=True)

#tmatches

# now matching the two objects where Inada and eftek. has the objects switched
tunique, tmatches, idx1, idx2 = match_tables_fib_2RA(the_whills,eftek2017,2)

for i, j in zip(idx1, idx2):
    the_whills.at[i, 'Sep'] = eftek2017.at[j, 'Sep']
    the_whills.at[i, 'Paper(s)'] += ' ; Eftekharzadeh+2017'
    the_whills.at[i, 'BibCode(s)'] += ' ; 2017MNRAS.468...77E' 
    the_whills.at[i, 'DOI(s)'] += ' ; https://doi.org/10.1093/mnras/stx412'

# dropping the matching indices from the eftek2017 table
eftek2017.drop(idx2, axis=0, inplace=True)
eftek2017.reset_index(drop=True, inplace=True)

the_whills = pd.concat([the_whills,eftek2017])

# verified


In [28]:
#the_whills = the_whills_test

In [29]:
#print(len(eftek2017))

In [30]:
#tunique, tmatches, idx1, idx2 = match_tables_fib_2RA(the_whills,eftek2017,2)
#
#print(len(tmatches)) #there are no matches!
#
##the_whills = pd.concat([the_whills,more2016])
#
## there are 2 matches
#
#tmatches

In [31]:
# Agnello+2018 here

# S0332−0021 is listed as a contaminating QSO + QSO but only one redshift is listed.
# We need to reach out and ask if this is a projected pair or not

agnello2018 = pd.read_csv('Tables/Agnello2018/agnello2018.csv', sep=',')

# No distinct coordinates are provided. We need to reach out and request them.
# No separations are provided
# A0326 overlaps with Schecter. They also claim there is more overlap between Schecter and their NIQs but 
# I cannot find any other matching objects to support this notion

# all uncertain/unclear cases are included here along with their NIQs.

# Since the format of the catalog will require names for both components, we'll add in a column 'Name2' which for \
# these and similar targets will be duplicates of the first 'Name column'. Same goes for z2, etc. 
agnello2018['Name1'] = agnello2018['Name']
agnello2018['Name2'] = '-99'
agnello2018['z2'] = -99
agnello2018['z1_type'] = "spec"
agnello2018['z2_type'] = "-99"

# Now converting the naming convention to RA and Dec and adding some informative columns
#name_to_coords(agnello2018,agnello2018['Name'])

## Converting the coordinates
#coordconvert = SkyCoord(ra = agnello2018['RA'], dec = agnello2018['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
agnello2018['RA1_deg'] = agnello2018['RA(J2000)']
agnello2018['Dec1_deg'] = agnello2018['Dec(J2000)']

## Convert Ra and Dec from degrees to sexagesimal format
coords = SkyCoord(ra=agnello2018['RA1_deg']*u.degree, dec=agnello2018['Dec1_deg']*u.degree, frame='icrs')
agnello2018['RA1'] = coords.ra.to_string(u.hour, sep=':', precision=2)
agnello2018['Dec1'] = coords.dec.to_string(u.deg, sep=':', precision=2)

## Concatenate to form J_format using a loop
#agnello2018['Name1'] = ["J" + ra + dec for ra, dec in zip(ra_format, dec_format)]

# Adding in a second set of coordinates for the 'secondary'
agnello2018['RA2'] = -99
agnello2018['Dec2'] = -99

agnello2018['RA2_deg'] = -99
agnello2018['Dec2_deg'] = -99

# Adding details about the coordinates
agnello2018['Equinox1'] = "J2000"
agnello2018['Coordinate_waveband1'] = "Optical"
agnello2018['Coordinate_Source1'] = "SDSS"

agnello2018['Equinox2'] = "-99"
agnello2018['Coordinate_waveband2'] = "-99"
agnello2018['Coordinate_Source2'] = "-99"

agnello2018['System Type'] = 'Dual AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
agnello2018['Brightness1'] = -100
agnello2018['Brightness_band1'] = -100
agnello2018['Brightness_type1'] = -100

agnello2018['Brightness2'] = -100
agnello2018['Brightness_band2'] = -100
agnello2018['Brightness_type2'] = -100

#agnello2018['Sep'] = 3 # arcseconds

#agnello2018['Sep(kpc)'] = agnello2018['Sep']*((cosmo.arcsec_per_kpc_proper(agnello2018['z']))**(-1))

agnello2018['dV'] = -99

# Adding information about the paper and the selection method
#agnello2018['Selection Method'] = "-99" #DPSELs
#agnello2018['Confirmation Method'] = "-99"
agnello2018['Paper(s)'] = "Agnello+2018"
agnello2018['BibCode(s)'] = "2018MNRAS.475.2086A"
agnello2018['DOI(s)'] = "https://doi.org/10.1093/mnras/stx3226"

agnello2018['Notes'] = 'Nearly identical quasars and no lens galaxy detected. For S0332 need to verify that this is not a dual quasar.'

# And dropping any columns that we don't need....
#agnello2018.drop(labels=['e_z','Del1','e_Del1','Del2','e_Del2','FOIII1','e_FOIII1','FOIII2','e_FOIII2','D'],\
#              axis=1, inplace=True)

#agnello2018



In [32]:
# add in agnello2018 now...

tunique, tmatches, idx1, idx2 = match_tables_fib(the_whills,agnello2018,20)

#print(len(tmatches)) #there are no matches!

the_whills = pd.concat([the_whills,agnello2018])

# verified


In [33]:
# Spiniello+2018
# Distinct coordinates are not provided.

spiniello2018 = pd.read_csv('Tables/spiniello2018/spiniello2018.csv', sep=',')

# No distinct coordinates are provided. We need to reach out and request them.

# Since the format of the catalog will require names for both components, we'll add in a column 'Name2' which for \
# these and similar targets will be duplicates of the first 'Name column'. Same goes for z2, etc. 

# We include all objects from Table 2 and 3, except for this entry since they are confirmed to be stars:
# KIDS0834-0139,08:34:40,-01:39:08,DIA,3.0,Not a lens. TNG Spectroscopy reveals that the two objects are stars.

spiniello2018['Name1'] = spiniello2018['Name']
spiniello2018['Name2'] = '-99'
spiniello2018['z1'] = -99
spiniello2018['z2'] = -99
spiniello2018['z1_type'] = "-99"
spiniello2018['z2_type'] = "-99"

# Now converting the naming convention to RA and Dec and adding some informative columns
spiniello2018['RA1'] = spiniello2018['RA']
spiniello2018['Dec1'] = spiniello2018['Dec']

# Converting the coordinates
coordconvert = SkyCoord(ra = spiniello2018['RA1'], dec = spiniello2018['Dec1'], frame='icrs', unit = (u.hourangle, u.deg))
spiniello2018['RA1_deg'] = coordconvert.ra.degree
spiniello2018['Dec1_deg'] = coordconvert.dec.degree

# Adding in a second set of coordinates for the 'secondary'
spiniello2018['RA2'] = -99
spiniello2018['Dec2'] = -99

spiniello2018['RA2_deg'] = -99
spiniello2018['Dec2_deg'] = -99

# Adding details about the coordinates
spiniello2018['Equinox1'] = "J2000"
spiniello2018['Coordinate_waveband1'] = "Optical"
spiniello2018['Coordinate_Source1'] = "SDSS"

spiniello2018['Equinox2'] = "-99"
spiniello2018['Coordinate_waveband2'] = "-99"
spiniello2018['Coordinate_Source2'] = "-99"

spiniello2018['System Type'] = 'Dual AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
spiniello2018['Brightness1'] = -100
spiniello2018['Brightness_band1'] = -100
spiniello2018['Brightness_type1'] = -100

spiniello2018['Brightness2'] = -100
spiniello2018['Brightness_band2'] = -100
spiniello2018['Brightness_type2'] = -100

#spiniello2018['Sep'] = 3 # arcseconds

#spiniello2018['Sep(kpc)'] = spiniello2018['Sep']*((cosmo.arcsec_per_kpc_proper(spiniello2018['z']))**(-1))

#spiniello2018['dV'] = (2.99e+5)*((1+spiniello2018['z'])**2 - (1+spiniello2018['z2'])**2)/((1+spiniello2018['z'])**2+(1+spiniello2018['z2'])**2)
## dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
#spiniello2018['Selection Method'] = "-99" #DPSELs
#spiniello2018['Confirmation Method'] = "-99"
spiniello2018['Paper(s)'] = "Spiniello+2018"
spiniello2018['BibCode(s)'] = "2018MNRAS.480.1163S"
spiniello2018['DOI(s)'] = "https://doi.org/10.1093/mnras/sty1923"

spiniello2018['Notes'] = ' '

# And dropping any columns that we don't need....
#spiniello2018.drop(labels=['e_z','Del1','e_Del1','Del2','e_Del2','FOIII1','e_FOIII1','FOIII2','e_FOIII2','D'],\
#              axis=1, inplace=True)


#spiniello2018



In [34]:
# now adding in the Spiniello2018 table...

tunique, tmatches, idx1, idx2 = match_tables_fib(the_whills,spiniello2018,20)

#print(len(tmatches)) 

for i, j in zip(idx1, idx2):
    the_whills.at[i, 'Paper(s)'] += ' ; Spiniello+2018'
    the_whills.at[i, 'BibCode(s)'] += ' ; 2018MNRAS.480.1163S' 
    the_whills.at[i, 'DOI(s)'] += ' ; https://doi.org/10.1093/mnras/sty1923'

# dropping the matching indices from the spiniello2018 table
spiniello2018.drop(idx2, axis=0, inplace=True)
spiniello2018.reset_index(drop=True, inplace=True)


the_whills = pd.concat([the_whills,spiniello2018])

#tmatches

# verified


In [35]:
# Lemon+2018
# I do include a few objects listed a inconclusive, but not the object that is listed as quasar+star? 
# or the inconclusive pair that is labeled as likely stars
# inconclusife pairs from table 2 aqre not ioncluded here

# No distinct coordinates are provided by Lemon. We need to reach out and request them.

# Here we're loading in the double-peaked emission line galaxy catalog of Wang+2009
lemon2018 = pd.read_csv('Tables/Lemon2018/Lemon2018_t2.csv', sep=',')
# table1.dat is a modified version of Wang's catalog in which I've added a duplicate row for each target
# since all of these are candidate dual AGN systems

# Since the format of the catalog will require names for both components, we'll add in a column 'Name2' which for \
# these and similar targets will be duplicates of the first 'Name column'. Same goes for z2, etc. 

lemon2018['Name1'] = lemon2018['Name']
lemon2018['Name2'] = '-99'
lemon2018['z1'] = lemon2018['z']
lemon2018['z2'] = lemon2018['z'] # these are NIQs with nearly identical redshifts
lemon2018['z1_type'] = "spec"
lemon2018['z2_type'] = "spec"

# Convert Ra and Dec from degrees to sexagesimal format
coords = SkyCoord(ra=lemon2018['RA']*u.degree, dec=lemon2018['Dec']*u.degree, frame='icrs')
lemon2018['RA1'] = coords.ra.to_string(u.hour, sep=':', precision=2)
lemon2018['Dec1'] = coords.dec.to_string(u.deg, sep=':', precision=2)
lemon2018['RA1_deg'] = lemon2018['RA']
lemon2018['Dec1_deg'] = lemon2018['Dec']

lemon2018['RA2'] = -99
lemon2018['Dec2'] = -99

lemon2018['RA2_deg'] = -99
lemon2018['Dec2_deg'] = -99

# Adding details about the coordinates
lemon2018['Equinox1'] = "J2000"
lemon2018['Coordinate_waveband1'] = "Optical"
lemon2018['Coordinate_Source1'] = "SDSS"

lemon2018['Equinox2'] = "-99"
lemon2018['Coordinate_waveband2'] = "-99"
lemon2018['Coordinate_Source2'] = "-99"

lemon2018['System Type'] = 'Dual AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
lemon2018['Brightness1'] = -100
lemon2018['Brightness_band1'] = -100
lemon2018['Brightness_type1'] = -100

lemon2018['Brightness2'] = -100
lemon2018['Brightness_band2'] = -100
lemon2018['Brightness_type2'] = -100

lemon2018['Sep'] = lemon2018['Sep_as']

#lemon2018['Sep(kpc)'] = lemon2018['Sep']*((cosmo.arcsec_per_kpc_proper(lemon2018['z']))**(-1))

#lemon2018['dV'] = (2.99e+5)*((1+lemon2018['z'])**2 - (1+lemon2018['z2'])**2)/((1+lemon2018['z'])**2+(1+lemon2018['z2'])**2)
# dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
#lemon2018['Selection Method'] = "-99" #DPSELs
#lemon2018['Confirmation Method'] = "-99"
lemon2018['Paper(s)'] = "Lemon+2018"
lemon2018['BibCode(s)'] = "2018MNRAS.479.5060L"
lemon2018['DOI(s)'] = "https://doi.org/10.1093/mnras/sty911"

lemon2018['Notes'] = 'Nearly identical quasars and no lens galaxy detected.'

# And dropping any columns that we don't need....
#lemon2018.drop(labels=['e_z','Del1','e_Del1','Del2','e_Del2','FOIII1','e_FOIII1','FOIII2','e_FOIII2','D'],\
#              axis=1, inplace=True)

#lemon2018


In [36]:
# adding lemon2018 here

tunique, tmatches, idx1, idx2 = match_tables_fib(the_whills,lemon2018,20)

print(len(tmatches)) # 1 matches

for i, j in zip(idx1, idx2):
    the_whills.at[i, 'Paper(s)'] += ' ; Lemon+2018'
    the_whills.at[i, 'BibCode(s)'] += ' ; 2018MNRAS.479.5060L' 
    the_whills.at[i, 'DOI(s)'] += ' ; https://doi.org/10.1093/mnras/sty911'

# dropping the matching indices from the lemon2018 table
lemon2018.drop(idx2, axis=0, inplace=True)
lemon2018.reset_index(drop=True, inplace=True)

the_whills = pd.concat([the_whills,lemon2018])

#tmatches

# verified


2


In [37]:
#the_whills_test


In [38]:
# Lemon+2019
# They quote 5 NIQs but I count only four in the table. Need to ask about missing one.
# I include two onbjects classified simply as 'inconclusive'


# The method of selection is shown for each candidate: WD, WT, WQ = ALLWISE double, triple, quad; uW = unWISE model fitting; MQ = Milliquas;

# No distinct coordinates are provided by Lemon. We need to reach out and request them.

# Here we're loading in the double-peaked emission line galaxy catalog of Wang+2009
lemon2019 = pd.read_csv('Tables/Lemon2019/Lemon2019_t4.csv', sep=',')
# table1.dat is a modified version of Wang's catalog in which I've added a duplicate row for each target
# since all of these are candidate dual AGN systems

# Since the format of the catalog will require names for both components, we'll add in a column 'Name2' which for \
# these and similar targets will be duplicates of the first 'Name column'. Same goes for z2, etc. 

lemon2019['Name1'] = lemon2019['Name']
lemon2019['Name2'] = '-99'
lemon2019['z1'] = lemon2019['z']
lemon2019['z2'] = lemon2019['z'] # these are NIQs or inconclusives, so we're adopting identical redshifts
lemon2019['z1_type'] = "spec"
lemon2019['z2_type'] = "spec"

# Convert Ra and Dec from degrees to sexagesimal format
coords = SkyCoord(ra=lemon2019['RA']*u.degree, dec=lemon2019['Dec']*u.degree, frame='icrs')
lemon2019['RA1'] = coords.ra.to_string(u.hour, sep=':', precision=2)
lemon2019['Dec1'] = coords.dec.to_string(u.deg, sep=':', precision=2)
lemon2019['RA1_deg'] = lemon2019['RA']
lemon2019['Dec1_deg'] = lemon2019['Dec']

lemon2019['RA2'] = -99
lemon2019['Dec2'] = -99

lemon2019['RA2_deg'] = -99
lemon2019['Dec2_deg'] = -99

# Adding details about the coordinates
lemon2019['Equinox1'] = "J2000"
lemon2019['Coordinate_waveband1'] = "Optical"
lemon2019['Coordinate_Source1'] = "SDSS"

lemon2019['Equinox2'] = "-99"
lemon2019['Coordinate_waveband2'] = "-99"
lemon2019['Coordinate_Source2'] = "-99"

lemon2019['System Type'] = 'Dual AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
lemon2019['Brightness1'] = -100
lemon2019['Brightness_band1'] = -100
lemon2019['Brightness_type1'] = -100

lemon2019['Brightness2'] = -100
lemon2019['Brightness_band2'] = -100
lemon2019['Brightness_type2'] = -100

lemon2019['Sep'] = -99 # arcseconds

#lemon2019['Sep(kpc)'] = lemon2019['Sep']*((cosmo.arcsec_per_kpc_proper(lemon2019['z']))**(-1))

#lemon2019['dV'] = (2.99e+5)*((1+lemon2019['z'])**2 - (1+lemon2019['z2'])**2)/((1+lemon2019['z'])**2+(1+lemon2019['z2'])**2)
# dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
#lemon2019['Selection Method'] = "-99" #DPSELs
#lemon2019['Confirmation Method'] = "-99"
lemon2019['Paper(s)'] = "Lemon+2019"
lemon2019['BibCode(s)'] = "2019MNRAS.483.4242L"
lemon2019['DOI(s)'] = "https://doi.org/10.1093/mnras/sty3366"

lemon2019['Notes'] = 'Nearly identical quasars and no lens galaxy detected.'

# And dropping any columns that we don't need....
#lemon2019.drop(labels=['e_z','Del1','e_Del1','Del2','e_Del2','FOIII1','e_FOIII1','FOIII2','e_FOIII2','D'],\
#              axis=1, inplace=True)

# for J2316+0610, mark redshift type as -99
# for J1307+0642, mark redshift type as -99
for index, row in the_whills.iterrows():
    if row['Name1']=='J2316+0610':
        the_whills.at[index, 'z1_type'] = '-99'
        the_whills.at[index, 'z2_type'] = '-99'
    if row['Name1']=='J1307+0642':
        the_whills.at[index, 'z1_type'] = '-99'
        the_whills.at[index, 'z2_type'] = '-99'

#lemon2019


In [39]:
# adding lemon 2019 here...

tunique, tmatches, idx1, idx2 = match_tables_fib(the_whills,lemon2019,20)

#print(len(tmatches)) # 1 matches

for i, j in zip(idx1, idx2):
    the_whills.at[i, 'Paper(s)'] += ' ; Lemon+2019'
    the_whills.at[i, 'BibCode(s)'] += ' ; 2019MNRAS.479.5060L' 
    the_whills.at[i, 'DOI(s)'] += ' ; https://doi.org/10.1093/mnras/sty911'

# dropping the matching indices from the lemon2019 table
lemon2019.drop(idx2, axis=0, inplace=True)
lemon2019.reset_index(drop=True, inplace=True)

the_whills = pd.concat([the_whills,lemon2019])

#tmatches

# verified

In [40]:
#the_whills

In [41]:
# Now loading in the tables for Rusu+2019

rusu2019t1 = pd.read_csv('Tables/Rusu2019/Rusu2019_t1.csv', sep=',') # this contains the 91 candidates they identified
rusu2019t1b = pd.read_csv('Tables/Rusu2019/Rusu2019_t1b.csv', sep=',') # This contains previously confirmed candidate (lens? - unclear in the text) as well as rejected ones

# In table 2, confirmation_flag==2 are candidates that have been ruled out becased on GAIA properties or spectroscopy
# confirmation_flag==1 are confirmed lenses or binary quasars

# Here we are removing those irrelevant rejected candidates:
rusu2019t1b = rusu2019t1b[rusu2019t1b['Confirmation_flag']<2]
# And now removing things mnually based on objects that were previously discovered 
# It seems that we only need to include the first entry, which is a pair of SDSS QSOs
# It looks like all other entries are accounted for in our bibliography (I'm not going to try to match this \
# against my other tables; there's way too much going on here... maybe I can come back and fix this later)

# In their table 1, we're including all of their candidates

rusu2019t1['Name1'] = rusu2019t1['Name']
rusu2019t1['Name2'] = '-99'
rusu2019t1['z1'] = -99
rusu2019t1['z2'] = -99
rusu2019t1['z1_type'] = "-99"
rusu2019t1['z2_type'] = "-99"

rusu2019t1['Selection Method'] = 'Optical Imaging'

# Converting the coordinates
rusu2019t1['RA1_deg'] = rusu2019t1['RAdeg']#coordconvert.ra.degree
rusu2019t1['Dec1_deg'] = rusu2019t1['DEdeg']#coordconvert.dec.degree
#
# Convert Ra and Dec from degrees to sexagesimal format
coords = SkyCoord(ra=rusu2019t1['RA1_deg']*u.degree, dec=rusu2019t1['Dec1_deg']*u.degree, frame='icrs')
rusu2019t1['RA1'] = coords.ra.to_string(u.hour, sep=':', precision=2)
rusu2019t1['Dec1'] = coords.dec.to_string(u.deg, sep=':', precision=2)

rusu2019t1['RA2'] = -99
rusu2019t1['Dec2'] = -99

rusu2019t1['RA2_deg'] = -99
rusu2019t1['Dec2_deg'] = -99

# Adding details about the coordinates
rusu2019t1['Equinox1'] = "J2000"
rusu2019t1['Coordinate_waveband1'] = "-99"
rusu2019t1['Coordinate_Source1'] = "-99"

rusu2019t1['Equinox2'] = "-99"
rusu2019t1['Coordinate_waveband2'] = "-99"
rusu2019t1['Coordinate_Source2'] = "-99"

rusu2019t1['System Type'] = 'Dual AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
rusu2019t1['Brightness1'] = -100
rusu2019t1['Brightness_band1'] = -100
rusu2019t1['Brightness_type1'] = -100

rusu2019t1['Brightness2'] = -100
rusu2019t1['Brightness_band2'] = -100
rusu2019t1['Brightness_type2'] = -100

rusu2019t1['Sep'] = rusu2019t1['Sep(as)'] # arcseconds

#rusu2019t1['Sep(kpc)'] = rusu2019t1['Sep']*((cosmo.arcsec_per_kpc_proper(rusu2019t1['z']))**(-1))

#rusu2019t1['dV'] = (2.99e+5)*((1+rusu2019t1['z'])**2 - (1+rusu2019t1['z2'])**2)/((1+rusu2019t1['z'])**2+(1+rusu2019t1['z2'])**2)
# dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
#rusu2019t1['Selection Method'] = "-99" #DPSELs
#rusu2019t1['Confirmation Method'] = "-99"
rusu2019t1['Paper(s)'] = "Rusu+2019"
rusu2019t1['BibCode(s)'] = "2019MNRAS.486.4987R"
rusu2019t1['DOI(s)'] = "https://doi.org/10.1093/mnras/stz1142"

rusu2019t1['Notes'] = ' '

# And dropping any columns that we don't need....
#rusu2019t1.drop(labels=['e_z','Del1','e_Del1','Del2','e_Del2','FOIII1','e_FOIII1','FOIII2','e_FOIII2','D'],\
#              axis=1, inplace=True)


#rusu2019t1



In [42]:
#rusu2019t1b

rusu2019t1b['Name1'] = rusu2019t1b['Name']
rusu2019t1b['Name2'] = '-99'
rusu2019t1b['z1'] = -99
rusu2019t1b['z2'] = -99
rusu2019t1b['z1_type'] = "-99"
rusu2019t1b['z2_type'] = "-99"

# Converting the coordinates
#coordconvert = SkyCoord(ra = rusu2019t1b['RA'], dec = rusu2019t1b['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
rusu2019t1b['RA1_deg'] = rusu2019t1b['RAdeg']#coordconvert.ra.degree
rusu2019t1b['Dec1_deg'] = rusu2019t1b['DEdeg']#coordconvert.dec.degree

# Convert Ra and Dec from degrees to sexagesimal format
coords = SkyCoord(ra=rusu2019t1b['RA1_deg']*u.degree, dec=rusu2019t1b['Dec1_deg']*u.degree, frame='icrs')
rusu2019t1b['RA1'] = coords.ra.to_string(u.hour, sep=':', precision=2)
rusu2019t1b['Dec1'] = coords.dec.to_string(u.deg, sep=':', precision=2)

rusu2019t1b['RA2'] = -99
rusu2019t1b['Dec2'] = -99

rusu2019t1b['RA2_deg'] = -99
rusu2019t1b['Dec2_deg'] = -99

# Adding details about the coordinates
rusu2019t1b['Equinox1'] = "J2000"
rusu2019t1b['Coordinate_waveband1'] = "-99"
rusu2019t1b['Coordinate_Source1'] = "-99"

rusu2019t1b['Equinox2'] = "-99"
rusu2019t1b['Coordinate_waveband2'] = "-99"
rusu2019t1b['Coordinate_Source2'] = "-99"

rusu2019t1b['System Type'] = 'Dual AGN Candidate'
rusu2019t1b['Selection Method'] = 'Optical Imaging'

# Adding in some columns that we'll population via a Simbad or Ned search later
rusu2019t1b['Brightness1'] = -100
rusu2019t1b['Brightness_band1'] = -100
rusu2019t1b['Brightness_type1'] = -100

rusu2019t1b['Brightness2'] = -100
rusu2019t1b['Brightness_band2'] = -100
rusu2019t1b['Brightness_type2'] = -100

rusu2019t1b['Sep'] = rusu2019t1b['Sep(as)'] # arcseconds

#rusu2019t1b['Sep(kpc)'] = rusu2019t1b['Sep']*((cosmo.arcsec_per_kpc_proper(rusu2019t1b['z']))**(-1))

#rusu2019t1b['dV'] = (2.99e+5)*((1+rusu2019t1b['z'])**2 - (1+rusu2019t1b['z2'])**2)/((1+rusu2019t1b['z'])**2+(1+rusu2019t1b['z2'])**2)
# dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
#rusu2019t1b['Selection Method'] = "-99" #DPSELs
#rusu2019t1b['Confirmation Method'] = "-99"
rusu2019t1b['Paper(s)'] = "Rusu+2019"
rusu2019t1b['BibCode(s)'] = "2019MNRAS.486.4987R"
rusu2019t1b['DOI(s)'] = "https://doi.org/10.1093/mnras/stz1142"

rusu2019t1b['Notes'] = ' '

# And dropping any columns that we don't need....
#rusu2019t1b.drop(labels=['e_z','Del1','e_Del1','Del2','e_Del2','FOIII1','e_FOIII1','FOIII2','e_FOIII2','D'],\
#              axis=1, inplace=True)

rusu2019t1b = rusu2019t1b.loc[[0]]
# HERE I AM ONLY INCLUDING THE SEEMINGLY UNIQUE QSO PAIR NOR IDENTIFIED BY OTHERS BEFORE

#rusu2019t1b


In [43]:
rusu2019 = pd.concat([rusu2019t1,rusu2019t1b])


In [44]:
# adding in Rusu2019 now...

tunique, tmatches, idx1, idx2 = match_tables_fib(the_whills,rusu2019,20)

print(len(tmatches)) # 4 matches

for i, j in zip(idx1, idx2):
    the_whills.at[i, 'Paper(s)'] += ' ; Rusu+2019'
    the_whills.at[i, 'BibCode(s)'] += ' ; 2019MNRAS.486.4987R' 
    the_whills.at[i, 'DOI(s)'] += ' ; https://doi.org/10.1093/mnras/stz1142'

# manually adding in the separation from Rusu for the object in Spiniello2-18 that's missing one
for index, row in the_whills.iterrows():
    if row['Name1']=='KIDS1217-0256':
        the_whills.at[index, 'Sep'] = 1.700

# dropping the matching indices from the rusu2019 table
rusu2019.drop(idx2, axis=0, inplace=True)
rusu2019.reset_index(drop=True, inplace=True)

the_whills = pd.concat([the_whills,rusu2019])

#tmatches

# 


8


In [45]:
#the_whills

In [46]:
# Lemon+2020
#Quasar pairs. Selection: G1: Gaia 1, G2: Gaia 2, V: variability, C: component fitting. NIQ stands for nearly identical quasar pair.

# No distinct coordinates are provided by Lemon. We need to reach out and request them.

# there were no inconclusive cases included in their tables this time, so no need to include any

# Here we're loading in the double-peaked emission line galaxy catalog of Wang+2009
lemon2020 = pd.read_csv('Tables/Lemon2020/Lemon2020_t2.csv', sep=',')
# table1.dat is a modified version of Wang's catalog in which I've added a duplicate row for each target
# since all of these are candidate dual AGN systems

# Since the format of the catalog will require names for both components, we'll add in a column 'Name2' which for \
# these and similar targets will be duplicates of the first 'Name column'. Same goes for z2, etc. 

lemon2020['Name1'] = lemon2020['Name']
lemon2020['Name2'] = '-99'
lemon2020['z1'] = lemon2020['z']
lemon2020['z2'] = lemon2020['z'] # these are nearly identical quasars with effectively identical redshifts
lemon2020['z1_type'] = "spec"
lemon2020['z2_type'] = "spec"

# Convert Ra and Dec from degrees to sexagesimal format
coords = SkyCoord(ra=lemon2020['RA(J2000)']*u.degree, dec=lemon2020['Dec(J2000)']*u.degree, frame='icrs')
lemon2020['RA1'] = coords.ra.to_string(u.hour, sep=':', precision=2)
lemon2020['Dec1'] = coords.dec.to_string(u.deg, sep=':', precision=2)
lemon2020['RA1_deg'] = lemon2020['RA(J2000)']
lemon2020['Dec1_deg'] = lemon2020['Dec(J2000)']

lemon2020['RA2'] = -99
lemon2020['Dec2'] = -99

lemon2020['RA2_deg'] = -99
lemon2020['Dec2_deg'] = -99

# Adding details about the coordinates
lemon2020['Equinox1'] = "J2000"
lemon2020['Coordinate_waveband1'] = "Optical"
lemon2020['Coordinate_Source1'] = "SDSS"

lemon2020['Equinox2'] = "-99"
lemon2020['Coordinate_waveband2'] = "-99"
lemon2020['Coordinate_Source2'] = "-99"

lemon2020['System Type'] = 'Dual AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
lemon2020['Brightness1'] = -100
lemon2020['Brightness_band1'] = -100
lemon2020['Brightness_type1'] = -100

lemon2020['Brightness2'] = -100
lemon2020['Brightness_band2'] = -100
lemon2020['Brightness_type2'] = -100

lemon2020['Sep'] = lemon2020['Sep_as']

#lemon2020['Sep(kpc)'] = lemon2020['Sep']*((cosmo.arcsec_per_kpc_proper(lemon2020['z']))**(-1))

#lemon2020['dV'] = (2.99e+5)*((1+lemon2020['z'])**2 - (1+lemon2020['z2'])**2)/((1+lemon2020['z'])**2+(1+lemon2020['z2'])**2)
# dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
#lemon2020['Selection Method'] = "-99" #DPSELs
#lemon2020['Confirmation Method'] = "-99"
lemon2020['Paper(s)'] = "Lemon+2020"
lemon2020['BibCode(s)'] = "2020MNRAS.494.3491L"
lemon2020['DOI(s)'] = "https://doi.org/10.1093/mnras/staa652"

lemon2020['Notes'] = 'Nearly identical quasars and no lens galaxy detected.'

# And dropping any columns that we don't need....
#lemon2020.drop(labels=['e_z','Del1','e_Del1','Del2','e_Del2','FOIII1','e_FOIII1','FOIII2','e_FOIII2','D'],\
#              axis=1, inplace=True)

#lemon2020


In [47]:
# adding in Lemon 2020 here

tunique, tmatches, idx1, idx2 = match_tables_fib(the_whills,lemon2020,20)

#print(len(tmatches)) # 2 matches

for i, j in zip(idx1, idx2):
    the_whills.at[i, 'z1'] = lemon2020.at[j, 'z1']
    the_whills.at[i, 'z2'] = lemon2020.at[j, 'z2']
    the_whills.at[i, 'z1_type'] = lemon2020.at[j, 'z1_type']
    the_whills.at[i, 'z2_type'] = lemon2020.at[j, 'z2_type']
    the_whills.at[i, 'Paper(s)'] += ' ; Lemon+2020'
    the_whills.at[i, 'BibCode(s)'] += ' ; 2020MNRAS.494.3491L' 
    the_whills.at[i, 'DOI(s)'] += ' ; https://doi.org/10.1093/mnras/staa652'
    the_whills.at[i, 'Notes'] += '. Nearly identical quasars and no lens galaxy detected.'

# I'm keeping the originally listed separations from eftek2017 and rusu2019

## dropping the matching indices from the lemon2020 table
lemon2020.drop(idx2, axis=0, inplace=True)
lemon2020.reset_index(drop=True, inplace=True)

the_whills = pd.concat([the_whills,lemon2020])

#tmatches



In [48]:
#the_whills

In [49]:
# Here we're adding in information from Hutsemekers+2020 
# This is a follow-up to the object SDSS J081830.46+060138.0 (J0818+0601)

hutsemekers2020 = ['SDSS J0818+0601A'] # 
for index, row in the_whills.iterrows():
    if row['Name1'] in hutsemekers2020:
        the_whills.at[index, 'Paper(s)'] += ' ; Hutsemekers+2020 '
        the_whills.at[index, 'BibCode(s)'] += ' ; 2020A&A...633A.101H' 
        the_whills.at[index, 'DOI(s)'] += ' ; https://doi.org/10.1051/0004-6361/201936973'
        the_whills.at[index, 'Notes']='Hutsemekers+ find in their spectropolarimetric obs that the BAL profiles are nearly identical between the two spectra. This strongly suppoprts the notion of a lens rather than a binary given how unlikely it is that to find two similar absorbers in the two AGNs. Microlensing appears to show two sources of continuum. An intervening absorption line system is observed around z=1 in both images but with different intensities. This absorption line system hints at the presence of a lens galaxy.'
        the_whills.at[index, 'System Type'] = 'Binary Quasar Candidate'

# we'll need to come back and use a flag of -0.5 or -1 for this target


In [50]:
#the_whills

In [51]:
# Anguita+2018
# Distinct coordinates are not provided.

anguita2018 = pd.read_csv('Tables/Anguita2018/Anguita2018_t2.csv', sep=',')

# No distinct coordinates are provided. We need to reach out and request them.
# Verify no overlap with Schecter+2017

# 0120 was also independently identified by Ostrovski+ but I can't find a paper by Ostrovski that examines this object
# 2141 may have been looked at by Hennawi+

# 0120 and 2141 show evidence in their spectra of having distinct quasar pairs 

# We've included their NIQs, the 'other' inconclusive cases, as well as one case of awhat they refer to as a \
# projected pair that might be physically associated

# Since the format of the catalog will require names for both components, we'll add in a column 'Name2' which for \
# these and similar targets will be duplicates of the first 'Name column'. Same goes for z2, etc. 

anguita2018['Name2'] = '-99'
#anguita2018['z2'] = '-99'
anguita2018['z1_type'] = "spec"
anguita2018['z2_type'] = "-99"

for index, row in anguita2018.iterrows():
    if (row['z2']==0):
        anguita2018.at[index, 'z2'] = -99

## Converting the coordinates
#coordconvert = SkyCoord(ra = anguita2018['RA'], dec = anguita2018['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
anguita2018['RA1'] = '-99'
anguita2018['Dec1'] = '-99'
anguita2018['RA1_deg'] = '-99'
anguita2018['Dec1_deg'] = '-99'
#
## Adding in a second set of coordinates for the 'secondary'
anguita2018['RA2'] = '-99'
anguita2018['Dec2'] = '-99'

anguita2018['RA2_deg'] = '-99'
anguita2018['Dec2_deg'] = '-99'

# Adding details about the coordinates
anguita2018['Equinox1'] = "J2000"
anguita2018['Coordinate_waveband1'] = "Optical"
anguita2018['Coordinate_Source1'] = "SDSS"

anguita2018['Equinox2'] = "J2000"
anguita2018['Coordinate_waveband2'] = "Optical"
anguita2018['Coordinate_Source2'] = "SDSS"

anguita2018['System Type'] = 'Dual AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
anguita2018['Brightness1'] = -100
anguita2018['Brightness_band1'] = -100
anguita2018['Brightness_type1'] = -100

anguita2018['Brightness2'] = -100
anguita2018['Brightness_band2'] = -100
anguita2018['Brightness_type2'] = -100

anguita2018['Sep'] = anguita2018['Sep_as']

#anguita2018['Sep(kpc)'] = anguita2018['Sep']*((cosmo.arcsec_per_kpc_proper(anguita2018['z']))**(-1))

anguita2018['dV'] = -99
for index, row in anguita2018.iterrows():
    if (row['z1']!=-99) & (row['z2']!=-99):
        anguita2018.at[index, 'dV'] = (2.99e+5)*((1+anguita2018.at[index,'z1'])**2 - (1+anguita2018.at[index,'z2'])**2)/((1+anguita2018.at[index,'z1'])**2+(1+anguita2018.at[index,'z2'])**2)
    else:
        anguita2018.at[index, 'dV'] = -99
# dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
#anguita2018['Selection Method'] = "-99" #DPSELs
#anguita2018['Confirmation Method'] = "-99"
anguita2018['Paper(s)'] = "Anguita+2018"
anguita2018['BibCode(s)'] = "2018MNRAS.480.5017A"
anguita2018['DOI(s)'] = "https://doi.org/10.1093/mnras/sty2172"

anguita2018['Notes'] = 'Nearly identical quasars and no lens galaxy detected. .'

# And dropping any columns that we don't need....
#anguita2018.drop(labels=['e_z','Del1','e_Del1','Del2','e_Del2','FOIII1','e_FOIII1','FOIII2','e_FOIII2','D'],\
#              axis=1, inplace=True)

#anguita2018



In [52]:
# adding in the anguita2018 table now...

# we can do a straight concatenation because there are no coordinates for us to match on

the_whills = pd.concat([the_whills,anguita2018])

# verified


In [53]:
# removing NaN values

the_whills.fillna('-99', inplace=True)


In [54]:
#the_whills

In [55]:
# now we're going to format the final table and remove columns we don't need

the_whills.drop(labels=['level_0','index','iSDSS','i_cor','theta_SDSS','theta_iband','Comment','Ref','imag1',\
                        'thetaSDSS1','Comment1','imag2','thetaSDSS2','Comment2','reference','Table_flag','Name',\
                        'i_band_psfmag(extcorr)','sep_as','i_band_psfmag(extcorr)1','i_band_psfmag(extcorr)2',\
                        'Sep_as','imag','theta','Com','Selection','Result','iA','iB','Sep_as1','Obs_stat1',\
                        'RAdeg1','DEdeg1','gmag1','Class1','QQ?','Sep_as2','Obs_stat2','RAdeg2','DEdeg2',\
                        'gmag2','Class2','RA(J2000)','Dec(J2000)','mag_i','class','Methods','Grade','z',\
                        'Gaia_G1','Gaia_G2','RAdeg','DEdeg','Num_comp','i_mag','Sep(as)','Rank','Gmag1',\
                        'Gmag2','Gmag3','Confirmation_flag','g'], axis=1, inplace=True)
                        

In [56]:
#the_whills

In [57]:
# and now we're finally saving the table

the_whills.to_csv('Duals_from_lens_searches.csv', sep=',', index=False)


In [58]:
the_whills

,z1,Selection Method,Confirmation Method,z2,Name1,Name2,z2_type,RA,Dec,RA1,Dec1,RA1_deg,Dec1_deg,RA2,Dec2,RA2_deg,Dec2_deg,Equinox1,Coordinate_waveband1,Coordinate_Source1,Equinox2,Coordinate_waveband2,Coordinate_Source2,System Type,Brightness1,Brightness_band1,Brightness_type1,Brightness2,Brightness_band2,Brightness_type2,Sep,dV,Paper(s),BibCode(s),DOI(s),Notes,z1_type
0,1.959,Fiber Optical Spectroscopy / Optical Spectrosc...,-99,-99.000,SDSSJ034801.20-070416.9,-99,-99,03:48:01.20,-07:04:16.9,03:48:01.20,-07:04:16.9,57.005,-7.071361,-99,-99,-99.0,-99.0,J2000,Optical,SDSS,-99,-99,-99,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,1.54,-99.0,Inada+2008,2008AJ....135..496I,https://doi.org/10.1088/0004-6256/135/2/496,-99,-99
1,1.404,Fiber Optical Spectroscopy / Optical Spectrosc...,-99,-99.000,SDSSJ072843.03+370834.9,-99,-99,07:28:43.03,+37:08:34.9,07:28:43.03,+37:08:34.9,112.179292,37.143028,-99,-99,-99.0,-99.0,J2000,Optical,SDSS,-99,-99,-99,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,1.34,-99.0,Inada+2008,2008AJ....135..496I,https://doi.org/10.1088/0004-6256/135/2/496,-99,-99
2,2.166,Fiber Optical Spectroscopy / Optical Spectrosc...,-99,-99.000,SDSSJ074352.62+245743.6,-99,-99,07:43:52.62,+24:57:43.6,07:43:52.62,+24:57:43.6,115.96925,24.962111,-99,-99,-99.0,-99.0,J2000,Optical,SDSS,-99,-99,-99,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,1.02,-99.0,Inada+2008,2008AJ....135..496I,https://doi.org/10.1088/0004-6256/135/2/496,-99,-99
3,1.696,Fiber Optical Spectroscopy / Optical Spectrosc...,-99,-99.000,SDSSJ083530.89+054240.7,-99,-99,08:35:30.89,+05:42:40.7,08:35:30.89,+05:42:40.7,128.878708,5.711306,-99,-99,-99.0,-99.0,J2000,Optical,SDSS,-99,-99,-99,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.99,-99.0,Inada+2008,2008AJ....135..496I,https://doi.org/10.1088/0004-6256/135/2/496,-99,-99
4,0.627,Fiber Optical Spectroscopy / Optical Spectrosc...,Optical Fiber Spectroscopy,0.626,SDSSJ084710.40-001302.6,-99,-99,08:47:10.40,-00:13:02.6,08:47:10.40,-00:13:02.6,131.793333,-0.217389,-99,-99,-99.0,-99.0,J2000,Optical,SDSS,-99,-99,-99,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.95,183.830293,Inada+2008 ; Spiniello+2018,2008AJ....135..496I ; 2018MNRAS.480.1163S,https://doi.org/10.1088/0004-6256/135/2/496 ; ...,-99,-99
5,0.894,Fiber Optical Spectroscopy / Optical Spectrosc...,-99,-99.000,SDSSJ085122.37+472249.0,-99,-99,08:51:22.37,+47:22:49.0,08:51:22.37,+47:22:49.0,132.843208,47.380278,-99,-99,-99.0,-99.0,J2000,Optical,SDSS,-99,-99,-99,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,1.47,-99.0,Inada+2008,2008AJ....135..496I,https://doi.org/10.1088/0004-6256/135/2/496,-99,-99
6,1.993,Fiber Optical Spectroscopy / Optical Spectrosc...,Optical Fiber Spectroscopy,1.994,SDSSJ093207.15+072251.3,-99,-99,09:32:07.15,+07:22:51.3,09:32:07.15,+07:22:51.3,143.029792,7.380917,-99,-99,-99.0,-99.0,J2000,Optical,SDSS,-99,-99,-99,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,1.42,-99.883077,Inada+2008,2008AJ....135..496I,https://doi.org/10.1088/0004-6256/135/2/496,-99,-99
7,0.619,Fiber Optical Spectroscopy / Optical Spectrosc...,-99,-99.000,SDSSJ095324.39+570319.5,-99,-99,09:53:24.39,+57:03:19.5,09:53:24.39,+57:03:19.5,148.351625,57.055417,-99,-99,-99.0,-99.0,J2000,Optical,SDSS,-99,-99,-99,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,2.57,-99.0,Inada+2008 ; Rusu+2019,2008AJ....135..496I ; 2019MNRAS.486.4987R,https://doi.org/10.1088/0004-6256/135/2/496 ; ...,-99,-99
8,2.052,Fiber Optical Spectroscopy / Optical Spectrosc...,-99,-99.000,SDSSJ100229.46+444942.7,-99,-99,10:02:29.46,+44:49:42.7,10:02:29.46,+44:49:42.7,150.62275,44.828528,-99,-99,-99.0,-99.0,J2000,Optical,SDSS,-99,-99,-99,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.72,-99.0,Inada+2008,2008AJ....135..496I,https://doi.org/10.1088/0004-6256/135/2/496,-99,-99
9,1.138,Fiber Optical Spectroscopy / Optical Spectrosc...,-99,-99.000,SDSSJ100327.37+595804.0,-99,-99,10:03:27.37,+59:58:04.0,10:03:27.37,+59:58:04.0,150.864042,59.967778,-99,-99,-99.0,-99.0,J2000,Optical,SDSS,-99

In [59]:
inada2010t3_forlater

,Name1,z1,Selection Method,Confirmation Method,i_band_psfmag(extcorr)1,Name2,z2,i_band_psfmag(extcorr)2,Sep_as,Comment,z1_type,z2_type,RA1,Dec1,RA2,Dec2,RA1_deg,Dec1_deg,RA2_deg,Dec2_deg,Equinox1,Coordinate_waveband1,Coordinate_Source1,Equinox2,Coordinate_waveband2,Coordinate_Source2,System Type,Brightness1,Brightness_band1,Brightness_type1,Brightness2,Brightness_band2,Brightness_type2,Sep,dV,Paper(s),BibCode(s),DOI(s),Notes
0,SDSSJ032029.75+000650.0,1.704,Fiber Optical Spectroscopy / Optical Spectrosc...,NaN,19.05,SDSSJ032030.90+000658.0,-99.000,20.34,18.99,No lens object,spec,-99,03:20:29.75,+00:06:50.0,03:20:30.90,+00:06:58.0,50.123958,0.113889,50.128750,0.116111,J2000,Optical,SDSS,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,18.99,-99.000000,Inada+2010,2010AJ....140..403I,https://doi.org/10.1088/0004-6256/140/2/403,-99
1,SDSSJ034347.00-000706.5,1.975,Fiber Optical Spectroscopy / Optical Spectrosc...,NaN,18.85,SDSSJ034347.48-000658.9,-99.000,19.30,10.49,No lens object,spec,-99,03:43:47.00,-00:07:06.5,03:43:47.48,-00:06:58.9,55.945833,-0.118472,55.947833,-0.116361,J2000,Optical,SDSS,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,10.49,-99.000000,Inada+2010,2010AJ....140..403I,https://doi.org/10.1088/0004-6256/140/2/403,-99
2,SDSSJ072653.68+394706.9,1.599,Fiber Optical Spectroscopy / Optical Spectrosc...,NaN,18.92,SDSSJ072653.66+394710.6,-99.000,18.84,3.69,No lens object,spec,-99,07:26:53.68,+39:47:06.9,07:26:53.66,+39:47:10.6,111.723667,39.785250,111.723583,39.786278,J2000,Optical,SDSS,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,3.69,-99.000000,Inada+2010,2010AJ....140..403I,https://doi.org/10.1088/0004-6256/140/2/403,-99
3,SDSSJ074550.99+503423.1,1.737,Fiber Optical Spectroscopy / Optical Spectrosc...,NaN,18.98,SDSSJ074550.78+503430.3,-99.000,19.84,7.44,No lens object,spec,-99,07:45:50.99,+50:34:23.1,07:45:50.78,+50:34:30.3,116.462458,50.573083,116.461583,50.575083,J2000,Optical,SDSS,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,7.44,-99.000000,Inada+2010,2010AJ....140..403I,https://doi.org/10.1088/0004-6256/140/2/403,-99
4,SDSSJ080932.70+193847.2,1.670,Fiber Optical Spectroscopy / Optical Spectrosc...,NaN,18.49,SDSSJ080931.82+193849.3,-99.000,19.28,12.68,No lens object,spec,-99,08:09:32.70,+19:38:47.2,08:09:31.82,+19:38:49.3,122.386250,19.646444,122.382583,19.647028,J2000,Optical,SDSS,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,12.68,-99.000000,Inada+2010,2010AJ....140..403I,https://doi.org/10.1088/0004-6256/140/2/403,-99
5,SDSSJ081210.96+070826.2,1.862,Fiber Optical Spectroscopy / Optical Spectrosc...,NaN,17.65,SDSSJ081210.93+070838.7,-99.000,16.89,12.39,No lens object,spec,-99,08:12:10.96,+07:08:26.2,08:12:10.93,+07:08:38.7,123.045667,7.140611,123.045542,7.144083,J2000,Optical,SDSS,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,12.39,-99.000000,Inada+2010,2010AJ....140..403I,https://doi.org/10.1088/0004-6256/140/2/403,-99
6,SDSSJ084109.72+250200.2,1.227,Fiber Optical Spectroscopy / Optical Spectrosc...,NaN,19.06,SDSSJ084109.88+250216.0,-99.000,20.13,15.89,no lens object,spec,-99,08:41:09.72,+25:02:00.2,08:41:09.88,+25:02:16.0,130.290500,25.033389,130.291167,25.037778,J2000,Optical,SDSS,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,15.89,-99.000000,Inada+2010,2010AJ....140..403I,https://doi.org/10.1088/0004-6256/140/2/403,-99
7,SDSSJ084359.79+073229.7,2.175,Fiber Optical Spectroscopy / Optical Spectrosc...,NaN,19.02,SDSSJ084359.89+073215.9,-99.000,20.28,13.97,no lens object,spec,-99,08:43:59.79,+07:32:29.7,08:43:59.89,+07:32:15.9,130.999125,7.541583,130.999542,7.537750,J2000,Optical,SDSS,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,13.97,-99.000000,Inada+2010,2010AJ....140..403I,https://doi.org/10.1088/0004-6256/140/2/403,-99
8,SDSSJ085705.91+270149.0,1.419,Fiber Optical Spectroscopy / Optical Spectrosc...,NaN,19.08,SDSSJ085706.14+270147.6,-99.0

In [60]:
the_whills.columns

Index(['z1', 'Selection Method', 'Confirmation Method', 'z2', 'Name1', 'Name2',
       'z2_type', 'RA', 'Dec', 'RA1', 'Dec1', 'RA1_deg', 'Dec1_deg', 'RA2',
       'Dec2', 'RA2_deg', 'Dec2_deg', 'Equinox1', 'Coordinate_waveband1',
       'Coordinate_Source1', 'Equinox2', 'Coordinate_waveband2',
       'Coordinate_Source2', 'System Type', 'Brightness1', 'Brightness_band1',
       'Brightness_type1', 'Brightness2', 'Brightness_band2',
       'Brightness_type2', 'Sep', 'dV', 'Paper(s)', 'BibCode(s)', 'DOI(s)',
       'Notes', 'z1_type'],
      dtype='object')